# Setting Up Your Lab Environment

This guide shows you how to setup a Hashi environment for testing features in Consul, Vault, and Nomad.

Things to note:
* If you use Enterprise binaries
  * Enterprise binaries (`+ent`) need to be licensed - set in `docker-compose*.yml`
  * Consul 1.9, Nomad 1.0, Vault 1.7 has a starter license of 6 hours.
  * Consul 1.10+, Nomad 1.1+, and Vault 1.8+ requires a license file or it won't start
  * `Prem` images have their licenses baked in.


## Prerequisites

### Set Key Variables for your environment

Customize `CONSUL_DC` and `CONSUL_DC_2` if desired.

In [206]:
export CONSUL_DC=west CONSUL_DC_2=east
export COMPOSE_PROJECT_NAME=hashi
#// compose file - files on the right take precedence
export COMPOSE_FILE=docker-compose.yml:docker-compose-proxy.yml:docker-compose-vault.yml:docker-compose-hashi.yml

In [207]:
printf "$CONSUL_DC \n$CONSUL_DC_2 \n$COMPOSE_PROJECT_NAME \n$COMPOSE_FILE"

west 
east 
hashi 
docker-compose.yml:docker-compose-proxy.yml:docker-compose-vault.yml:docker-compose-hashi.yml

* `CONSUL_DC*` - is used for Consul config files, docker-compose files, and more.
* `COMPOSE_FILE` - specifies the docker-compose files to work with

### Install software

#### Hashi software

Customize the versions, architecture, and os for your environment. It's currently set for Ubuntu on Pi.

In [6]:
VAULT_VER=1.8.5+ent #// +ent for enterprise
CONSUL_VER=1.10.4+ent
NOMAD_VER=1.2.2 #1.1.7+ent
ARCH=arm64 #// arm64
OS=linux #// darwin, linux

curl -o /tmp/vault.zip \
  https://releases.hashicorp.com/vault/${VAULT_VER}/vault_${VAULT_VER}_${OS}_${ARCH}.zip
curl -o /tmp/consul.zip \
  https://releases.hashicorp.com/consul/${CONSUL_VER}/consul_${CONSUL_VER}_${OS}_${ARCH}.zip
curl -o /tmp/nomad.zip \
  https://releases.hashicorp.com/nomad/${NOMAD_VER}/nomad_${NOMAD_VER}_${OS}_${ARCH}.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.4M  100 63.4M    0     0  17.8M      0  0:00:03  0:00:03 --:--:-- 17.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37.8M  100 37.8M    0     0  20.3M      0  0:00:01  0:00:01 --:--:-- 20.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.2M  100 29.2M    0     0  18.5M      0  0:00:01  0:00:01 --:--:-- 18.5M


In [8]:
for bin in consul vault nomad; do
sudo unzip -od /usr/local/bin /tmp/${bin}.zip && ${bin} version
done

Archive:  /tmp/consul.zip
  inflating: /usr/local/bin/consul   
Consul v1.10.4+ent
Revision cbb8cb74
Protocol 2 spoken by default, understands 2 to 3 (agent will automatically use protocol >2 when speaking to compatible agents)

Archive:  /tmp/vault.zip
  inflating: /usr/local/bin/vault    
  inflating: /usr/local/bin/EULA.txt  
  inflating: /usr/local/bin/TermsOfEvaluation.txt  
Vault v1.8.5+ent (10abdf02c159597fd916260e795c5dd480d4fb18)
Archive:  /tmp/nomad.zip
  inflating: /usr/local/bin/nomad    
Nomad v1.2.2 (78b8c171a211f967a8b297a88a7e844b3543f2b0)


In [12]:
for bin in consul vault nomad; do
${bin} -autocomplete-install || true
done

Error executing CLI: 1 error occurred:
	* already installed in /home/ubuntu/.bashrc


Error executing CLI: 1 error occurred:
	* already installed in /home/ubuntu/.bashrc


Error executing CLI: 1 error occurred:
	* already installed in /home/ubuntu/.bashrc




Ubuntu Linux - UNDER DEVELOPMENT - NOT WORKING FOR ARM64

In [3]:
ARCH=arm64
# curl -fsSL https://apt.releases.hashicorp.com/gpg | sudo apt-key add -
sudo apt-add-repository \
    "deb [arch=${ARCH}] https://apt.releases.hashicorp.com $(lsb_release -cs) main" \
    && sudo apt-get update #&& sudo apt-get install nomad

Get:1 https://apt.releases.hashicorp.com focal InRelease [9495 B]
Hit:2 http://ports.ubuntu.com/ubuntu-ports focal InRelease                     
Hit:3 http://ppa.launchpad.net/ansible/ansible/ubuntu focal InRelease          
Hit:4 http://ports.ubuntu.com/ubuntu-ports focal-updates InRelease       
Hit:5 http://ports.ubuntu.com/ubuntu-ports focal-backports InRelease     
Get:6 https://apt.releases.hashicorp.com focal/main arm64 Packages [1066 B]
Hit:7 https://apt.syncthing.net syncthing InRelease                           
Hit:8 http://ports.ubuntu.com/ubuntu-ports focal-security InRelease
Fetched 10.6 kB in 3s (3949 B/s)
Reading package lists... Done
Hit:1 https://apt.releases.hashicorp.com focal InRelease
Hit:2 http://ports.ubuntu.com/ubuntu-ports focal InRelease                     
Hit:3 http://ppa.launchpad.net/ansible/ansible/ubuntu focal InRelease          
Hit:4 http://ports.ubuntu.com/ubuntu-ports focal-updates InRelease       
Hit:5 http://ports.ubuntu.com/ubuntu-ports focal-

#### Other Software

In [ ]:
sudo apt install -qq unzip python3-pip

#### Docker and Docker Compose

* Ubuntu and Raspbian
  * https://dev.to/elalemanyo/how-to-install-docker-and-docker-compose-on-raspberry-pi-1mo

In [ ]:
#// docker
curl -sSL https://get.docker.com | sh
sudo usermod -aG docker ${USER}
sudo systemctl enable docker
#// docker-compose
sudo pip3 install docker-composesudo pip3 install docker-compose

* Mac - https://docs.docker.com/desktop/mac/install/

## Consul Setup - Primary

Create needed directories.

In [ ]:
mkdir -p consul/config
mkdir -p consul/cert/{server,client}

### Generate Consul Gossip Encryption Key

Generate encryption key for Gossip - UDP; same key for all agents; [more info](https://learn.hashicorp.com/consul/security-networking/agent-encryption)

In [13]:
CONSUL_KEY=$(consul keygen) && echo $CONSUL_KEY

Y+rrmAn0c9R7MLIf/eRNOjvglJA+z9dY/uqyqitaB0E=


Sample Output: `qDOPBEr+/oUVeOFQOnVypxwDaHzLrD+lvjo5vCEBbZ0=`

### Create CA and Certs for RPC Encryption

#### Create Certificate Authority

Create Certificate Authority

In [15]:
consul tls ca create || true

consul-agent-ca.pem already exists.


Output
```
==> Saved consul-agent-ca.pem
==> Saved consul-agent-ca-key.pem
```

Copy CA Public Key to shared `client` and `server` folders.

In [24]:
for dir in client server; do
cp -v consul-agent-ca.pem consul/cert/${dir}/
done

'consul-agent-ca.pem' -> 'consul/cert/client/consul-agent-ca.pem'
'consul-agent-ca.pem' -> 'consul/cert/server/consul-agent-ca.pem'


#### Create Server and Client Certificates

Create server certificate and move it to shared `server` folder.

In [25]:
consul tls cert create -server -dc ${CONSUL_DC}
mv ${CONSUL_DC}-server-consul-*.pem consul/cert/server/

==> WARNING: Server Certificates grants authority to become a
    server and access all state in the cluster including root keys
    and all ACL tokens. Do not distribute them to production hosts
    that are not server nodes. Store them as securely as CA keys.
==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved west-server-consul-0.pem
==> Saved west-server-consul-0-key.pem


Output
```
==> WARNING: Server Certificates grants authority to become a
    server and access all state in the cluster including root keys
    and all ACL tokens. Do not distribute them to production hosts
    that are not server nodes. Store them as securely as CA keys.
==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved west-server-consul-0.pem
==> Saved west-server-consul-0-key.pem
```

In [26]:
consul tls cert create -server -dc ${CONSUL_DC_2}
mv ${CONSUL_DC_2}-server-consul-*.pem consul/cert/server/

==> WARNING: Server Certificates grants authority to become a
    server and access all state in the cluster including root keys
    and all ACL tokens. Do not distribute them to production hosts
    that are not server nodes. Store them as securely as CA keys.
==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved east-server-consul-0.pem
==> Saved east-server-consul-0-key.pem


Output
```
==> WARNING: Server Certificates grants authority to become a
    server and access all state in the cluster including root keys
    and all ACL tokens. Do not distribute them to production hosts
    that are not server nodes. Store them as securely as CA keys.
==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved east-server-consul-0.pem
==> Saved east-server-consul-0-key.pem
```

Create client certificate and move it to shared `client` folder.

In [27]:
consul tls cert create -client -dc ${CONSUL_DC} && \
  mv ${CONSUL_DC}-client-consul-*.pem consul/cert/client

==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved west-client-consul-0.pem
==> Saved west-client-consul-0-key.pem


Output
```
==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved west-client-consul-0.pem
==> Saved west-client-consul-0-key.pem
```

In [28]:
consul tls cert create -client -dc ${CONSUL_DC_2} && \
  mv ${CONSUL_DC_2}-client-consul-*.pem consul/cert/client

==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved east-client-consul-0.pem
==> Saved east-client-consul-0-key.pem


### Create Consul Configs

#### Consul Server Configuration

Create Core Consul config - Server

Using a generic configuration. Using CLI parameters to customize at run time

In [30]:
# for i in {0..5}; do
tee consul/config/server.hcl <<-EOF
# datacenter  = "${CONSUL_DC}" # in CLI
# node_name   = "ConsulServer${i}" # in CLI or use hostname
bind_addr   = "0.0.0.0" #default
client_addr = "0.0.0.0" #default 127.0.0.1
data_dir    = "/consul/data"
log_level   = "DEBUG"

encrypt     = "${CONSUL_KEY}"
ca_file     = "/consul/cert/consul-agent-ca.pem"
cert_file   = "/consul/cert/${CONSUL_DC}-server-consul-0.pem"
key_file    = "/consul/cert/${CONSUL_DC}-server-consul-0-key.pem"
verify_incoming = true
verify_outgoing = true
verify_server_hostname = true

# server           =  true # in CLI
bootstrap_expect = 3
retry_join  = [ "consul-server-0", "consul-server-1", "consul-server-2" ]
ui_config { enabled = true } 

#// 5 is default multiplier
performance {
  raft_multiplier = 2 #// fast but not too fast
}

discovery_max_stale = "5s"

telemetry {
    prometheus_retention_time = "8h",
    disable_hostname = true
}

connect {
    enabled = true
}

enable_local_script_checks = true
EOF
# done

# datacenter  = "west" # in CLI
# node_name   = "ConsulServer" # in CLI or use hostname
bind_addr   = "0.0.0.0" #default
client_addr = "0.0.0.0" #default 127.0.0.1
data_dir    = "/consul/data"
log_level   = "DEBUG"

encrypt     = "Y+rrmAn0c9R7MLIf/eRNOjvglJA+z9dY/uqyqitaB0E="
ca_file     = "/consul/cert/consul-agent-ca.pem"
cert_file   = "/consul/cert/west-server-consul-0.pem"
key_file    = "/consul/cert/west-server-consul-0-key.pem"
verify_incoming = true
verify_outgoing = true
verify_server_hostname = true

# server           =  true # in CLI
ui_config { enabled = true } 
bootstrap_expect = 3
retry_join  = [ "consul-server-0", "consul-server-1", "consul-server-2" ]

#// 5 is default multiplier
performance {
  raft_multiplier = 2 #// fast but not too fast
}

discovery_max_stale = "5s"

telemetry {
    prometheus_retention_time = "8h",
    disable_hostname = true
}

connect {
    enabled = true
}

enable_local_script_checks = true


The config is generic-ish. The command in `docker-compose` file will specify more flags.

#### Consul Client Configuration

Create Core Consul config - Client

In [31]:
tee consul/config/client.hcl <<-EOF
# datacenter  = "dc1" # in CLI
# node_name   = "ConsulServer${i}" # in CLI
bind_addr   = "0.0.0.0" #default
client_addr = "0.0.0.0" #default 127.0.0.1
data_dir    = "/consul/data"

encrypt     = "${CONSUL_KEY}"
ca_file     = "/consul/cert/consul-agent-ca.pem"
cert_file   = "/consul/cert/${CONSUL_DC}-client-consul-0.pem"
key_file    = "/consul/cert/${CONSUL_DC}-client-consul-0-key.pem"
verify_incoming = true
verify_outgoing = true
verify_server_hostname = true

ui               = true
retry_join  = [ "consul-server-0", "consul-server-1", "consul-server-2" ]

discovery_max_stale = "5s"

telemetry {
    prometheus_retention_time = "8h",
    disable_hostname = true
}

connect {
    enabled = true
}

enable_local_script_checks = true
EOF

# datacenter  = "dc1" # in CLI
# node_name   = "ConsulServer" # in CLI
bind_addr   = "0.0.0.0" #default
client_addr = "0.0.0.0" #default 127.0.0.1
data_dir    = "/consul/data"

encrypt     = "Y+rrmAn0c9R7MLIf/eRNOjvglJA+z9dY/uqyqitaB0E="
ca_file     = "/consul/cert/consul-agent-ca.pem"
cert_file   = "/consul/cert/west-client-consul-0.pem"
key_file    = "/consul/cert/west-client-consul-0-key.pem"
verify_incoming = true
verify_outgoing = true
verify_server_hostname = true

ui               = true
retry_join  = [ "consul-server-0", "consul-server-1", "consul-server-2" ]

discovery_max_stale = "5s"

telemetry {
    prometheus_retention_time = "8h",
    disable_hostname = true
}

connect {
    enabled = true
}

enable_local_script_checks = true


#### Consul ACL Configuration

Create Consul config for misc features eg `acl`, `performance multiplier`, etc

In [101]:
cat > consul/config/acl.hcl << EOF
# acl = {
#   enabled = true
#   default_policy = "deny"
#   down_policy    = "extend-cache"
#   enable_token_persistence = true
#   # tokens = {
#   #   master = "49792521-8362-f878-5a32-7405f1783838"
#   # }
# }
EOF

In [ ]:
for i in {0..2}; do
docker-compose restart consul-server-${i}
sleep 3
done

Restarting consul-server-0 ... 
Restarting consul-server-1 ... mdone
Restarting consul-server-2 ... mdone
arting consul-server-2 ... done

In [ ]:
consul members
consul operator raft list-peers

Error getting peers: Failed to retrieve raft configuration: Unexpected response code: 403 (rpc error making call: Permission denied)


: 1

If ACLs are enabled correctly, the leader's logs will contain the following warning and info messages.

In [69]:
docker-compose logs --tail=100 | grep -i acl | grep INFO

consul-server-1    | 2021-12-07T23:06:39.111Z [INFO]  agent.server: initializing acls
consul-server-1    | 2021-12-07T23:06:39.113Z [INFO]  agent.leader: started routine: routine="legacy ACL token upgrade"
consul-server-1    | 2021-12-07T23:06:39.113Z [INFO]  agent.leader: started routine: routine="acl token reaping"


##### Create the bootstrap token

In [70]:
consul acl bootstrap

AccessorID:       697f9314-b11d-4013-73ee-b0882297fc7b
SecretID:         1eb8a07d-3b5a-9069-2673-e031a48851a7
Namespace:        default
Description:      Bootstrap Token (Global Management)
Local:            false
Create Time:      2021-12-07 23:11:28.792795908 +0000 UTC
Policies:
   00000000-0000-0000-0000-000000000001 - global-management



**VERIFY** - The logs should contain the following log message.

In [78]:
docker-compose logs --tail=100 | grep "ACL bootstrap"
docker-compose logs --tail=100 | grep "/v1/acl/bootstrap"

consul-server-0    | 2021-12-07T23:11:28.896Z [DEBUG] agent.http: Request finished: method=PUT url=/v1/acl/bootstrap from=10.5.0.1:49258 latency=104.639052ms
consul-server-1    | 2021-12-07T23:11:28.894Z [INFO]  agent.server.acl: ACL bootstrap completed


In [84]:
echo "#==> Without Token - will see no output"
consul members
echo "#==> With Token"
CONSUL_HTTP_TOKEN=1eb8a07d-3b5a-9069-2673-e031a48851a7 consul members

#==> Without Token - will see no output
#==> With Token
Node             Address         Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301   alive   server  1.9.11+ent  2         west  <all>
App1             10.5.0.12:8301  alive   client  1.9.11+ent  2         west  <default>


##### Configure Environment Variables

In [85]:
export CONSUL_HTTP_TOKEN=1eb8a07d-3b5a-9069-2673-e031a48851a7
# export CONSUL_CACERT=/etc/training/consul/consul-agent-ca.pem
# export CONSUL_CLIENT_CERT=/etc/training/consul/<dc-name>-<server/client>-consul-<cert-number>.pem
# export CONSUL_CLIENT_KEY=/etc/training/consul/<dc-name>-<server/client>-consul-<cert-number>-key.pem

##### Apply the Bootstrap Token to the Agents

In [92]:
for i in {0..2}; do
docker exec -i consul-server-${i} sh <<EOF
hostname
consul acl set-agent-token agent $CONSUL_HTTP_TOKEN
EOF
done

consul-server-0
consul acl set-agent-token agent 1eb8a07d-3b5a-9069-2673-e031a48851a7
consul-server-1
consul acl set-agent-token agent 1eb8a07d-3b5a-9069-2673-e031a48851a7
consul-server-2
consul acl set-agent-token agent 1eb8a07d-3b5a-9069-2673-e031a48851a7


ACL token "agent" set successfully

#### Validate Consul Configuration

In [33]:
consul validate consul/config/ || true

Config validation failed: 'bootstrap_expect > 0' requires 'server = true'


If you see this: `Config validation failed: 'bootstrap_expect > 0' requires 'server = true'`, you can ignore. We specify this from the command line.

In [87]:
consul members
consul operator raft list-peers

Node             Address         Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301   alive   server  1.9.11+ent  2         west  <all>
App1             10.5.0.12:8301  alive   client  1.9.11+ent  2         west  <default>
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-1  fcbab83a-8df1-bd3e-b8c1-aa4a1b4330d8  10.5.0.3:8300  leader    true   3
consul-server-2  2aa94199-c554-c021-c5a8-2f26d858c863  10.5.0.4:8300  follower  true   3
consul-server-0  a3bb7571-5fab-c616-94ad-c79aae44a575  10.5.0.2:8300  follower  true   3


Check on the UI

1. Go to http://192.168.17.101:8500
1. Then go to ACL and enter the token.
1. Go to `Nodes` tab and verify you can see the nodes.


### Start Consul docker-compose up

We will now bring up the three Consul servers and one client for our first Datacenter. I use `--force-recreate` to have Docker recreate the containers. This is handy for a fresh start when testing code bits.

In [35]:
#// Check your docker-compose configuration
docker-compose config

networks:
  vpcbr:
    driver: bridge
    ipam:
      config:
      - subnet: 10.5.0.0/16
services:
  cadvisor:
    container_name: cadvisor
    devices:
    - /dev/kmsg:/dev/kmsg
    image: raymondmm/cadvisor
    networks:
      vpcbr:
        ipv4_address: 10.5.0.11
    ports:
    - published: 8080
      target: 8080
    restart: on-failure
    volumes:
    - /dev/disk:/dev/disk:ro
    - /:/rootfs:ro
    - /sys:/sys:ro
    - /sys/fs/cgroup:/sys/fs/cgroup:ro
    - /var/lib/docker:/var/lib/docker:ro
    - /var/run:/var/run:rw
  consul-agent-1:
    command: agent -datacenter=west -retry-join consul-server-0 -client 0.0.0.0 -node=App1
    container_name: consul-agent-1
    environment:
      CONSUL_LICENSE_PATH: /consul/cert/consul.hclic
    hostname: consul-agent-1
    image: hashicorp/consul-enterprise:1.9-ent
    networks:
      vpcbr:
        ipv4_address: 10.5.0.12
    volumes:
    - /media/code/hc_demos-jupyter/HashiStack/consul/cert/client:/consul/cert:rw
    - /media/code/hc_demo

In [102]:
export CONSUL_DC=west CONSUL_DC_2=east
docker-compose \
  up --force-recreate -d \
  consul-server-0 consul-server-1 consul-server-2 consul-agent-1 #\
  # && docker ps --format "table {{.Image}}\t{{.Names}}\t{{.Ports}}"

Recreating consul-server-0 ... 
Recreating consul-server-1 ... 
Recreating consul-server-2 ... 
Recreating consul-agent-1  ... 
eating consul-server-0 ... done

> NOTE: We specify only the containers we want to bring up. If you don't specify something, then everything comes up.

Verify containers are up

In [208]:
docker ps --format "table {{.Image}}\t{{.Names}}\t{{.Ports}}" -f name=consul

IMAGE                                 NAMES             PORTS
hashicorp/consul-enterprise:1.9-ent   consul-server-0   0.0.0.0:8300-8302->8300-8302/tcp, :::8300-8302->8300-8302/tcp, 0.0.0.0:8500->8500/tcp, 0.0.0.0:8301-8302->8301-8302/udp, :::8500->8500/tcp, :::8301-8302->8301-8302/udp, 0.0.0.0:8600->8600/tcp, :::8600->8600/tcp, 0.0.0.0:8600->8600/udp, :::8600->8600/udp
hashicorp/consul-enterprise:1.9-ent   consul-server-2   8300-8302/tcp, 8500/tcp, 8301-8302/udp, 8600/tcp, 8600/udp
hashicorp/consul-enterprise:1.9-ent   consul-server-1   8300-8302/tcp, 8500/tcp, 8301-8302/udp, 8600/tcp, 8600/udp


### Verify Consul

In [209]:
for i in {0..2}; do
docker logs consul-server-${i} | (head; tail -n 5)
done

==> Starting Consul agent...
           Version: '1.9.11+ent'
           Node ID: 'a3bb7571-5fab-c616-94ad-c79aae44a575'
         Node name: 'consul-server-0'
        Datacenter: 'west' (Segment: '<all>')
            Server: true (Bootstrap: false)
       Client Addr: [0.0.0.0] (HTTP: 8500, HTTPS: -1, gRPC: -1, DNS: 8600)
      Cluster Addr: 10.5.0.2 (LAN: 8301, WAN: 8302)
           Encrypt: Gossip: true, TLS-Outgoing: true, TLS-Incoming: true, Auto-Encrypt-TLS: false

2021-12-08T20:14:10.866Z [DEBUG] agent: Service in sync: service=default/vault:10.5.0.101:8200
2021-12-08T20:14:10.866Z [DEBUG] agent: Check in sync: check=default/vault:10.5.0.101:8200:vault-sealed-check
2021-12-08T20:14:10.866Z [DEBUG] agent: Check in sync: check=default/vault:10.5.0.103:8200:vault-sealed-check
2021-12-08T20:14:10.866Z [DEBUG] agent: Check in sync: check=default/vault:10.5.0.102:8200:vault-sealed-check
2021-12-08T20:14:10.866Z [DEBUG] agent.http: Request finished: method=PUT url=/v1/agent/check/pass/v

Quick check to make sure your Consul environment is running correctly.

In [210]:
printf "#==> List Members\n"
consul members
# curl http://127.0.0.1:8500/v1/agent/members | jq -c .[]
printf "\n#==> List Raft Peers\n"
consul operator raft list-peers
printf "\n#==> List services from Consul catalog\n"
consul catalog services

#==> List Members
Node             Address        Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301  alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301  alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301  alive   server  1.9.11+ent  2         west  <all>

#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-1  fcbab83a-8df1-bd3e-b8c1-aa4a1b4330d8  10.5.0.3:8300  follower  true   3
consul-server-0  a3bb7571-5fab-c616-94ad-c79aae44a575  10.5.0.2:8300  follower  true   3
consul-server-2  2aa94199-c554-c021-c5a8-2f26d858c863  10.5.0.4:8300  leader    true   3

#==> List services from Consul catalog
consul
vault


You should see something like the following.

* There should be three servers. `DC` should match

```#==> List Members
Node             Address        Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301  alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301  alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301  alive   server  1.9.11+ent  2         west  <all>
```

* There should be a leader and two followers.

```
#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-2  08f89457-d9db-b025-c65e-185246fe577c  10.5.0.4:8300  leader    true   3
consul-server-1  f4c7057f-83ec-11ac-2027-ca85eccfce89  10.5.0.3:8300  follower  true   3
consul-server-0  2c965ad0-5042-424c-259c-a5781d001d28  10.5.0.2:8300  follower  true   3
```

```
#==> List services from Consul catalog
consul
```

## Vault Setup - Primary

### Create Vault Configs

In [ ]:
# Create Vault Directories
for node in {1..5}; do
mkdir -p vault/config/vault_s${node}
mkdir -p vault/logs/vault_s${node}
done

In [ ]:
# Create Vault Server Config
for i in {1..3}; do
cat > vault/config/vault_s${i}/server${i}.hcl <<-EOF
# Note: this file will be re-written by script
api_addr     = "http://10.5.0.10${i}:8200"
cluster_addr = "https://10.5.0.10${i}:8201"
cluster_name = "${CONSUL_DC}"
disable_mlock = true

# Base Configuration
listener "tcp" {
  address = "0.0.0.0:8200"
  tls_disable = "true"
#   #tls_cert_file = "/etc/ssl/certs/vault-server.crt"
#   #tls_key_file  = "/etc/ssl/vault-server.key"
}

ui = "true"
log_level="INFO"

# Raft configuration
storage "raft" {
  path    = "/vault/file"
  node_id = "vault_s${i}"
  retry_join {
    leader_api_addr = "http://vault_s1:8200"
  }
  retry_join {
    leader_api_addr = "http://vault_s2:8200"
  }
  retry_join {
    leader_api_addr = "http://vault_s3:8200"
  }
}

service_registration "consul" {
  address = "consul-server-0:8500"
}

telemetry {
  prometheus_retention_time = "30s"
  disable_hostname          = true
}
# raw_storage_endpoint = true #//for debugging
EOF
done

### Vault docker-compose up

In [138]:
# Restart Vault Cluster
docker-compose up --force-recreate -d \
  vault_s1 vault_s2 vault_s3

Pulling vault_s1 (hashicorp/vault-enterprise:1.8.5_ent)...
1.8.5_ent: Pulling from hashicorp/vault-enterprise

1f2373af: Pulling fs layer
d3fe0bfd: Pulling fs layer
ddcfbd56: Pulling fs layer
10ade137: Pulling fs layer
Digest: sha256:53c9550fb3f0619fdeacec7787f3d4bf2e9d14caca874ba06b275b8e56ff8009 1.828kB/1.828kBB
Status: Downloaded newer image for hashicorp/vault-enterprise:1.8.5_ent
Creating vault_s1 ... 
Creating vault_s2 ... mdone
Creating vault_s3 ... 
ting vault_s3 ... done

In [139]:
docker ps --format "table {{.Image}}\t{{.Names}}\t{{.Ports}}" -f name=vault

IMAGE                                  NAMES      PORTS
hashicorp/vault-enterprise:1.8.5_ent   vault_s3   8200/tcp
hashicorp/vault-enterprise:1.8.5_ent   vault_s2   8200/tcp
hashicorp/vault-enterprise:1.8.5_ent   vault_s1   0.0.0.0:8200->8200/tcp, :::8200->8200/tcp


### Init Vault `init.sh`

In [140]:
export VAULT_ADDR=http://localhost:8200

In [141]:
printf "#==> Init vault_s1 \n"
#// Confirm that vault_s1 is listening on port 8200
while ! nc -w 1 127.0.0.1 8200 </dev/null; do sleep 1; done
time vault operator init -format=json -n 1 -t 1 > /tmp/vault.init

#==> Init vault_s1 

real	0m9.139s
user	0m0.193s
sys	0m0.073s


In [145]:
export VAULT_TOKEN_PRIMARY=$(jq -r '.root_token' /tmp/vault.init)
printf "\nRoot VAULT TOKEN is: $VAULT_TOKEN_PRIMARY \n"
printf "\n*** Please Run: export VAULT_TOKEN=${VAULT_TOKEN_PRIMARY} \n"
export unseal_key=$(jq -r '.unseal_keys_b64[0]' /tmp/vault.init)
printf "\nUnseal Key is: ${unseal_key}\n"
export VAULT_TOKEN=${VAULT_TOKEN_PRIMARY}


Root VAULT TOKEN is: s.6NzYUa3teopZQOn8IEwcGJrf 

*** Please Run: export VAULT_TOKEN=s.6NzYUa3teopZQOn8IEwcGJrf 

Unseal Key is: OEfAgNWOazR11RAgm+YM7qe86EuZfbwStQd4k7IhH3U=


### Unseal Vault `unseal.sh`

In [146]:
vault operator unseal ${unseal_key}

Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.8.5+ent
Storage Type            raft
Cluster Name            west
Cluster ID              a1c9ef41-6421-71dc-c8b9-bfe2500f3151
HA Enabled              true
HA Cluster              n/a
HA Mode                 standby
Active Node Address     <none>
Raft Committed Index    54
Raft Applied Index      54


In [150]:
while ! vault operator raft list-peers > /dev/null 2>&1 ; do 
  sleep 1; echo "waiting..."
done
vault operator raft autopilot state

for i in {2..3}; do
docker exec -i vault_s${i} sh <<EOM
printf "\n#==> Unsealing:\n"
hostname
export VAULT_ADDR=http://localhost:8200
vault operator unseal ${unseal_key}
EOM
done


real	0m0.217s
user	0m0.177s
sys	0m0.094s
Healthy:                      true
Failure Tolerance:            1
Leader:                       vault_s1
Voters:
   vault_s1
   vault_s2
   vault_s3
Servers:
   vault_s1
      Name:            vault_s1
      Address:         10.5.0.101:8201
      Status:          leader
      Node Status:     alive
      Healthy:         true
      Last Contact:    0s
      Last Term:       3
      Last Index:      116
   vault_s2
      Name:            vault_s2
      Address:         10.5.0.102:8201
      Status:          voter
      Node Status:     alive
      Healthy:         true
      Last Contact:    4.765709116s
      Last Term:       3
      Last Index:      114
   vault_s3
      Name:            vault_s3
      Address:         10.5.0.103:8201
      Status:          voter
      Node Status:     alive
      Healthy:         true
      Last Contact:    2.577983252s
      Last Term:       3
      Last Index:      116


#==> Unsealing:
vault_s2
Key       

### Verify Vault

In [171]:
printf "#==> Check token\n"
vault token lookup
printf "\n#==> Check status\n"
vault status

#==> Check token
Key                 Value
---                 -----
accessor            3j9LKfic9zPskvgdULWi1yin
creation_time       1638941274
creation_ttl        0s
display_name        root
entity_id           n/a
expire_time         <nil>
explicit_max_ttl    0s
id                  s.6NzYUa3teopZQOn8IEwcGJrf
meta                <nil>
num_uses            0
orphan              true
path                auth/token/root
policies            [root]
ttl                 0s
type                service

#==> Check status
Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.8.5+ent
Storage Type            raft
Cluster Name            west
Cluster ID              a1c9ef41-6421-71dc-c8b9-bfe2500f3151
HA Enabled              true
HA Cluster              https://10.5.0.101:8201
HA Mode                 active
Active Since      

In [153]:
printf "#==> List Peers\n"
vault operator raft list-peers
printf "\n#==> Show autopilot state\n"
vault operator raft autopilot state || true
printf "\n#==> Show autopilot settings\n"
vault operator raft autopilot get-config || true

#==> List Peers
Node        Address            State       Voter
----        -------            -----       -----
vault_s1    10.5.0.101:8201    leader      true
vault_s2    10.5.0.102:8201    follower    true
vault_s3    10.5.0.103:8201    follower    true

#==> Show autopilot state
Healthy:                      true
Failure Tolerance:            1
Leader:                       vault_s1
Voters:
   vault_s1
   vault_s2
   vault_s3
Servers:
   vault_s1
      Name:            vault_s1
      Address:         10.5.0.101:8201
      Status:          leader
      Node Status:     alive
      Healthy:         true
      Last Contact:    0s
      Last Term:       3
      Last Index:      190
   vault_s2
      Name:            vault_s2
      Address:         10.5.0.102:8201
      Status:          voter
      Node Status:     alive
      Healthy:         true
      Last Contact:    4.763409144s
      Last Term:       3
      Last Index:      188
   vault_s3
      Name:            vault_s3
      A

In [154]:
vault secrets list
vault read sys/license

Path          Type         Accessor              Description
----          ----         --------              -----------
cubbyhole/    cubbyhole    cubbyhole_ba694efd    per-token private secret storage
identity/     identity     identity_77262344     identity store
sys/          system       system_f11e4d28       system endpoints used for control, policy and debugging
WARNING! The following warnings were returned from Vault:

  * time left on license is 108h38m48s

  * The GET sys/license API is deprecated and will be removed in a future
  release, use sys/license/status instead.

Key                          Value
---                          -----
expiration_time              2021-12-12T18:14:11Z
features                     [HSM Performance Replication DR Replication MFA Sentinel Seal Wrapping Control Groups Performance Standby Namespaces KMIP Entropy Augmentation Transform Secrets Engine Lease Count Quotas Key Management Secrets Engine Automated Snapshots]
license_id             

In [155]:
# vault write sys/license text=@vault/config/vault.hclic && \
# vault read sys/license

Error writing data to sys/license: Error making API request.

URL: PUT http://localhost:8200/v1/sys/license
Code: 400. Errors:

* unable to update stored license. autoloading is in effect and force=true not provided


: 2

In [62]:
vault secrets enable kv
# vault write kv/game/account username=foo password=bar

Success! Enabled the kv secrets engine at: kv/


In [80]:
for i in {1..10}; do
vault secrets enable -path=kv-app-${i} kv
done

Success! Enabled the kv secrets engine at: kv-app-1/
Success! Enabled the kv secrets engine at: kv-app-2/
Success! Enabled the kv secrets engine at: kv-app-3/
Success! Enabled the kv secrets engine at: kv-app-4/
Success! Enabled the kv secrets engine at: kv-app-5/
Success! Enabled the kv secrets engine at: kv-app-6/
Success! Enabled the kv secrets engine at: kv-app-7/
Success! Enabled the kv secrets engine at: kv-app-8/
Success! Enabled the kv secrets engine at: kv-app-9/
Success! Enabled the kv secrets engine at: kv-app-10/


In [75]:
for i in {1..10}; do
vault write kv/game/account-${i} username=foo-${i} password=bar
vault read kv/game/account-${i} > /dev/null
vault kv put kv-peter/data/game/account-${i} username=foo-${i} password=bar
vault kv get kv-peter/data/game/account-${i} > /dev/null
done

Success! Data written to: kv/game/account-1
Key              Value
---              -----
created_time     2021-11-29T23:55:23.782877801Z
deletion_time    n/a
destroyed        false
version          1
Success! Data written to: kv/game/account-2
Key              Value
---              -----
created_time     2021-11-29T23:55:24.912923349Z
deletion_time    n/a
destroyed        false
version          1
Success! Data written to: kv/game/account-3
Key              Value
---              -----
created_time     2021-11-29T23:55:25.999924852Z
deletion_time    n/a
destroyed        false
version          1
Success! Data written to: kv/game/account-4
Key              Value
---              -----
created_time     2021-11-29T23:55:27.546036117Z
deletion_time    n/a
destroyed        false
version          1
Success! Data written to: kv/game/account-5
Key              Value
---              -----
created_time     2021-11-29T23:55:28.598907884Z
deletion_time    n/a
destroyed        false
version       

In [81]:
for i in {1..10}; do
vault kv put kv-app-${i}/game/account-${i} username=foo-${i} password=bar
vault kv get kv-app-${i}/game/account-${i} > /dev/null
done

Success! Data written to: kv-app-1/game/account-1
Success! Data written to: kv-app-2/game/account-2
Success! Data written to: kv-app-3/game/account-3
Success! Data written to: kv-app-4/game/account-4
Success! Data written to: kv-app-5/game/account-5
Success! Data written to: kv-app-6/game/account-6
Success! Data written to: kv-app-7/game/account-7
Success! Data written to: kv-app-8/game/account-8
Success! Data written to: kv-app-9/game/account-9
Success! Data written to: kv-app-10/game/account-10


In [77]:
for i in {1..10}; do
vault token create \
  -field=token \
  -policy prometheus-metrics
done

s.t5ZV9E5b8Z9G7LdkIZ0led4E
s.JEhWqOrvrzSSZDpdTKNs58yz
s.FLBjoGHKKDd7suGHyoBwKfbv
s.bVppGIIchBYLSF0bQun3cPqo
s.1aEjSwWjYrSEHSLwERzD6oPA
s.6zgytcpMKG6coU6B0mH8IfcH
s.I62QLOEElZ04RJgOD4oYhlIY
s.3xAPLCWtDuIf3kfmfEOhZG5K
s.JVh0dpAl929Ipa5RVQhL8cD8
s.xjr1GeIyqXCPxZa3sROBFdZ7


In [72]:
vault secrets list -detailed

Path          Plugin       Accessor              Default TTL    Max TTL    Force No Cache    Replication    Seal Wrap    External Entropy Access    Options           Description                                                UUID
----          ------       --------              -----------    -------    --------------    -----------    ---------    -----------------------    -------           -----------                                                ----
cubbyhole/    cubbyhole    cubbyhole_22f47194    n/a            n/a        false             local          false        false                      map[]             per-token private secret storage                           af34d191-ae70-e114-2e4a-b4f86e005124
identity/     identity     identity_a4672f41     system         system     false             replicated     false        false                      map[]             identity store                                             04892ae6-be98-5bdd-83d9-91c08067a453
kv-peter/     kv

## Monitoring

In this scenario, you will use Docker containers to deploy a Vault server, Prometheus monitoring, and a Grafana dashboard.

You will configure Vault to enable Prometheus metrics, and deploy the containers using the command line in a terminal session. You will also use the Grafana web interface to create a dashboard for visualizing metrics.

Begin the scenario by preparing your environment.

### Prerequisites

* [Vault Cluster](#Vault-Setup---Primary)

In [ ]:
mkdir -p grafana/provisioning/{datasources,dashboards} \
  grafana/dashboards prometheus

### Vault configuration

Prometheus metrics are not enabled by default. Setting the `prometheus_retention_time` to a non-zero value enables them.

```
telemetry {
  prometheus_retention_time = "1h"
  disable_hostname          = true
}
```

* `prometheus_retention_time = "1h"` retain in memory for 1 hour
* `disable_hostname = true` - do not emit Prometheus metrics prefixed with host names, which is not desirable in most cases
* Go to [telemetry parameters](https://www.vaultproject.io/docs/configuration/telemetry#telemetry-parameters) documentation for more details.


This configuration was already included in the prerequisite sections.

### Prometheus Configuration

#### Vault Integration

Prep vault token for Prometheus.

In [156]:
printf "$(jq -r '.root_token' /tmp/vault.init)" > prometheus/prometheus-token
cat prometheus/prometheus-token

s.6NzYUa3teopZQOn8IEwcGJrf

Define a prometheus-metrics ACL policy that grants read capabilities to the metrics endpoint.

In [157]:
vault policy write prometheus-metrics - << EOF
path "/sys/metrics" {
  capabilities = ["read"]
}
EOF

Success! Uploaded policy: prometheus-metrics


In [178]:
cat > prometheus/prometheus.yml << EOF
# Generated via hashistack playbook
global:
  scrape_interval: 5s #default 1m
  scrape_timeout: 3s #default 10s

scrape_configs:
  - job_name: services
    metrics_path: /metrics
    static_configs:
      - targets:
        - 'prometheus:9090'
  - job_name: node
    metrics_path: /metrics
    static_configs:
      - targets:
        - 'node-exporter:9100'

  - job_name: 'consul-server'
    metrics_path: '/v1/agent/metrics'
    params:
      format: ['prometheus']
    static_configs:
      - targets: 
        - 'consul-server-0:8500'
        - 'consul-server-1:8500'
        - 'consul-server-2:8500'
        - 'consul-server-3:8500'
        - 'consul-server-4:8500'
        - 'consul-server-5:8500'
        - 'consul-agent-1:8500'

  - job_name: 'tempo'
    static_configs:
      - targets: ['tempo:3100']

  - job_name: vault
    metrics_path: /v1/sys/metrics
    params:
      format: ['prometheus']
    scheme: http
    authorization:
      credentials_file: /etc/prometheus/prometheus-token
    # static_configs:
    #   - targets: ['vault_s1:8200','vault_s2:8200','vault_s3:8200']
    #// dynamic targets via Consul
    consul_sd_configs:
      - server: 'consul-server-0:8500'
        services: ['vault']

  #// Container monitoring
  - job_name: cadvisor
    scrape_interval: 5s
    metrics_path: '/metrics'
    static_configs:
    - targets:
      - cadvisor:8080

EOF

Prometheus Jobs
* `prometheus`
* `consul`
* `vault`
* `nomad` - need to add
* `cadvisor`

### Start Prometheus docker-compose up

In [158]:
# Restart Prometheus
docker-compose up --force-recreate -d prometheus

Pulling prometheus (prom/prometheus:v2.26.0)...
v2.26.0: Pulling from prom/prometheus

70554e62: Pulling fs layer
78624da1: Pulling fs layer
9d2e3edb: Pulling fs layer
4cf32fce: Pulling fs layer
f79db750: Pulling fs layer
0a06e80e: Pulling fs layer
481d3927: Pulling fs layer
75ff1768: Pulling fs layer
7f567836: Pulling fs layer
5b797911: Pulling fs layer
6eb21eef: Pulling fs layer
Digest: sha256:38d40a760569b1c5aec4a36e8a7f11e86299e9191b9233672a5d41296d8fa74e    721B/721B1kBB
Status: Downloaded newer image for prom/prometheus:v2.26.0
Creating prometheus ... 
ting prometheus ... done

In [159]:
docker logs -n 1 prometheus

level=info ts=2021-12-08T05:39:14.450Z caller=main.go:767 msg="Server is ready to receive web requests."


The log should contain an entry like this one.
```shell
level=info ts=2021-11-20T01:29:46.330Z caller=main.go:767 msg="Server is ready to receive web requests."
```

#### Validate Prometheus

* Go to UI - http://<prometheus_ip>:9090
    * ex http://192.168.17.101:9090
* Run of some the following queries

| Metric | Description | |
| --- | --- | --- |
| `consul_raft_apply` | | |
| cAdvisor | | |
| `container_start_time_seconds` | The start time of containers (in seconds). You can select for specific containers by name using the `name="<container_name>"` expression.
| `rate(container_cpu_usage_seconds_total{name="consul-server-0"}[1m])` | The cgroup's CPU usage in the last minute
| `container_memory_usage_bytes{name="redis"}` | The cgroup's total memory usage (in bytes)
| `rate(container_network_transmit_bytes_total[1m])` | [link](http://192.168.17.101:9090/graph?g0.range_input=1h&g0.expr=rate(container_network_transmit_bytes_total%5B1m%5D)&g0.tab=1) - Bytes transmitted over the network by the container per second in the last minute
| `rate(container_network_receive_bytes_total[1m])` | Bytes received over the network by the container per second in the last minute

Reload prometheus if config has been modified.

In [179]:
docker exec prometheus kill -HUP 1
# curl -X POST http://localhost:9090/-/reload

#### node-exporter

In [160]:
# Restart node-exporter
docker-compose up --force-recreate -d node-exporter

Pulling node-exporter (prom/node-exporter:latest)...
latest: Pulling from prom/node-exporter

4d4e3d30: Pulling fs layer
6a21b340: Pulling fs layer
Digest: sha256:f2269e73124dd0f60a7d19a2ce1264d33d08a985aed0ee6b0b89d0be470592cd 8.401MB/8.401MBB
Status: Downloaded newer image for prom/node-exporter:latest
Creating node-exporter ... 
ting node-exporter ... done

Verify the status of node-exporter by checking its logs.

In [168]:
docker logs -n5 node-exporter

ts=2021-12-08T05:58:35.398Z caller=node_exporter.go:115 level=info collector=vmstat
ts=2021-12-08T05:58:35.398Z caller=node_exporter.go:115 level=info collector=xfs
ts=2021-12-08T05:58:35.398Z caller=node_exporter.go:115 level=info collector=zfs
ts=2021-12-08T05:58:35.399Z caller=node_exporter.go:199 level=info msg="Listening on" address=:9100
ts=2021-12-08T05:58:35.399Z caller=tls_config.go:195 level=info msg="TLS is disabled." http2=false


Output
```shell
ts=2021-12-08T05:58:35.398Z caller=node_exporter.go:115 level=info collector=vmstat
ts=2021-12-08T05:58:35.398Z caller=node_exporter.go:115 level=info collector=xfs
ts=2021-12-08T05:58:35.398Z caller=node_exporter.go:115 level=info collector=zfs
ts=2021-12-08T05:58:35.399Z caller=node_exporter.go:199 level=info msg="Listening on" address=:9100
ts=2021-12-08T05:58:35.399Z caller=tls_config.go:195 level=info msg="TLS is disabled." http2=false
```

In [117]:
# Restart cAdvisor
docker-compose up --force-recreate -d cadvisor

Recreating cadvisor ... 
eating cadvisor ... done

### Grafana Configuration

In [ ]:
cat > grafana/datasource.yml << EOF
# config file version
apiVersion: 1

datasources:
- name: vault
  type: prometheus
  access: server
  orgId: 1
  url: http://10.42.74.110:9090
  password:
  user:
  database:
  basicAuth:
  basicAuthUser:
  basicAuthPassword:
  withCredentials:
  isDefault:
  jsonData:
     graphiteVersion: "1.1"
     tlsAuth: false
     tlsAuthWithCACert: false
  secureJsonData:
    tlsCACert: ""
    tlsClientCert: ""
    tlsClientKey: ""
  version: 1
  editable: true
EOF

In [21]:
cat > grafana/provisioning/datasources/datasource_peter.yml << EOF
# config file version
apiVersion: 1
# list of datasources to insert/update depending
# what's available in the database
datasources:
- name: Loki
  type: loki
  access: proxy
  orgId: 1
  url: http://loki:3100
  basicAuth: false
  isDefault: true
  version: 1
  editable: false
  apiVersion: 1
  jsonData:
    derivedFields:
      - datasourceUid: tempo
        matcherRegex: (?:traceID|trace_id)=(\w+)
        name: TraceID
        url: $${__value.raw}


EOF

In [32]:
cat > grafana/provisioning/datasources/datasource_prometheus.yml << EOF
# config file version
apiVersion: 1
# list of datasources to insert/update depending
# what's available in the database
datasources:
- name: Prometheus
  type: prometheus
  access: proxy
  orgId: 1
  url: http://prometheus:9090
  # basicAuth: false
  isDefault: false
  version: 1
  editable: true
  apiVersion: 1

EOF

In [ ]:
chmod 755 grafana/provisioning/datasources/datasource.yml
chmod -R 755 grafana/provisioning
chmod -R 755 grafana/dashboards

In [ ]:
ll grafana/dashboards consul/config/

* `grafana/dashboards:/var/lib/grafana/dashboards` - preconfigured dashboards

### Start Grafana docker-compose up

In [169]:
# Restart Grafana
docker-compose up --force-recreate -d grafana

Pulling grafana (grafana/grafana:7.5.3)...
7.5.3: Pulling from grafana/grafana

9e8acc52: Pulling fs layer
761b3f10: Pulling fs layer
800a8865: Pulling fs layer
b700ef54: Pulling fs layer
37471b52: Pulling fs layer
bb76b650: Pulling fs layer
Digest: sha256:88d5dec7f18a06c726211eefca40b04c58cc94e99819c5dead046633d535b82d  1.22kB/1.22kBBB
Status: Downloaded newer image for grafana/grafana:7.5.3
Creating grafana ... 
ting grafana ... done

Validate Grafana

1. Go to `http://<grafana_ip>:3000` ex http://192.168.17.101:3000

#### Generate Traffic

Install dns tools into Consul server

In [166]:
docker exec consul-server-0 apk add bind-tools

fetch https://dl-cdn.alpinelinux.org/alpine/v3.13/main/aarch64/APKINDEX.tar.gz
fetch https://dl-cdn.alpinelinux.org/alpine/v3.13/community/aarch64/APKINDEX.tar.gz
(1/17) Installing fstrm (0.6.0-r1)
(2/17) Installing krb5-conf (1.0-r2)
(3/17) Installing libcom_err (1.45.7-r0)
(4/17) Installing keyutils-libs (1.6.3-r0)
(5/17) Installing libverto (0.3.1-r1)
(6/17) Installing krb5-libs (1.18.4-r0)
(7/17) Installing json-c (0.15-r1)
(8/17) Installing libgcc (10.2.1_pre1-r3)
(9/17) Installing libstdc++ (10.2.1_pre1-r3)
(10/17) Installing libprotobuf (3.13.0-r2)
(11/17) Installing libprotoc (3.13.0-r2)
(12/17) Installing protobuf-c (1.3.3-r4)
(13/17) Installing libuv (1.40.0-r0)
(14/17) Installing xz-libs (5.2.5-r0)
(15/17) Installing libxml2 (2.9.12-r0)
(16/17) Installing bind-libs (9.16.20-r1)
(17/17) Installing bind-tools (9.16.20-r1)
Executing busybox-1.32.1-r6.trigger
OK: 28 MiB in 47 packages


Run Consul load script. **TODO** Need to add more.

In [167]:
docker exec -i consul-server-0 sh <<"EOM"
cat > test_dns.sh <<"EOF"
#!/bin/sh
i=0
while [ $i -lt 100 ];do
dig @consul-server-0 -p8600 +short consul.service.consul SRV >> /tmp/test_dns.out
dig @consul-server-1 -p8600 +short consul.service.consul SRV >> /tmp/test_dns.out
dig @consul-server-2 -p8600 +short consul.service.consul SRV >> /tmp/test_dns.out
dig @consul-server-0 -p8600 +short -x 10.5.0.2 -x 10.5.0.3 >> /tmp/test_dns.out
dig @consul-server-1 -p8600 +short -x 10.5.0.2 -x 10.5.0.3 >> /tmp/test_dns.out
dig @consul-server-2 -p8600 +short -x 10.5.0.2 -x 10.5.0.3 >> /tmp/test_dns.out
export CONSUL_HTTP_ADDR=http://consul-server-0:8500
consul kv put redis/config/connections ${i} > /dev/null
consul kv get redis/config/connections > /dev/null
export CONSUL_HTTP_ADDR=http://consul-server-1:8500
consul kv put redis/config/connections ${i} > /dev/null
consul kv get redis/config/connections > /dev/null
export CONSUL_HTTP_ADDR=http://consul-server-2:8500
consul kv put redis/config/connections ${i} > /dev/null
consul kv get redis/config/connections > /dev/null
echo ${i}.c >> /tmp/test_dns.out
sleep 1
i=$(( $i +1 ))
done
EOF
chmod +x test_dns.sh
nohup ./test_dns.sh &
EOM

Look at these charts.
* Transaction Times
* System Stats
* DNS


#### Customizations

Raft Election
* Visualization: Stat
  * Orientation: Horizontal; Color mode: Background

Raft Election
* Visualization: Stat
  * Orientation: Horizontal; Color mode: Background
Raft Commit Time
* consul_raft_commitTime
* Visualization: Stat
  * Orientation: Horizontal; Color mode: Background
* Thresholds - Base, 80, 160

Grafana Dashboards
* cadvisor - https://grafana.com/grafana/dashboards/193

# Vault Monitoring

Install dns tools into Consul server

In [170]:
docker exec vault_s1 \
    apk add bind-tools dig

fetch https://dl-cdn.alpinelinux.org/alpine/v3.14/main/aarch64/APKINDEX.tar.gz
fetch https://dl-cdn.alpinelinux.org/alpine/v3.14/community/aarch64/APKINDEX.tar.gz
ERROR: unable to select packages:
  dig (no such package):
    required by: world[dig]


: 1

Run Consul load script. **TODO** Need to add more.

In [167]:
docker exec -i consul-server-0 sh <<"EOM"
cat > test_dns.sh <<"EOF"
#!/bin/sh
i=0
while [ $i -lt 100 ];do
dig @consul-server-0 -p8600 +short consul.service.consul SRV >> /tmp/test_dns.out
dig @consul-server-1 -p8600 +short consul.service.consul SRV >> /tmp/test_dns.out
dig @consul-server-2 -p8600 +short consul.service.consul SRV >> /tmp/test_dns.out
dig @consul-server-0 -p8600 +short -x 10.5.0.2 -x 10.5.0.3 >> /tmp/test_dns.out
dig @consul-server-1 -p8600 +short -x 10.5.0.2 -x 10.5.0.3 >> /tmp/test_dns.out
dig @consul-server-2 -p8600 +short -x 10.5.0.2 -x 10.5.0.3 >> /tmp/test_dns.out
export CONSUL_HTTP_ADDR=http://consul-server-0:8500
consul kv put redis/config/connections ${i} > /dev/null
consul kv get redis/config/connections > /dev/null
export CONSUL_HTTP_ADDR=http://consul-server-1:8500
consul kv put redis/config/connections ${i} > /dev/null
consul kv get redis/config/connections > /dev/null
export CONSUL_HTTP_ADDR=http://consul-server-2:8500
consul kv put redis/config/connections ${i} > /dev/null
consul kv get redis/config/connections > /dev/null
echo ${i}.c >> /tmp/test_dns.out
sleep 1
i=$(( $i +1 ))
done
EOF
chmod +x test_dns.sh
nohup ./test_dns.sh &
EOM

Look at these charts.
* Transaction Times
* System Stats
* DNS


## Vault Performance Nodes

optional - Vault Performance Nodes
* main difference here is that it does not auto-join
* joining manually as non-voter from CLI
* https://learn.hashicorp.com/tutorials/vault/performance-standbys?in=vault/enterprise

In [ ]:
# Create Vault Server Config
for i in {4..5}; do
cat > vault/config/vault_s${i}/server${i}.hcl <<-EOF
# Note: this file will be re-written by script
api_addr     = "http://10.5.0.10${i}:8200"
cluster_addr = "https://10.5.0.10${i}:8201"
disable_mlock = true

# Base Configuration
listener "tcp" {
  address = "0.0.0.0:8200"
  tls_disable = "true"
}

ui = "true"
log_level="INFO"

# Raft configuration
storage "raft" {
  path    = "/vault/file"
  node_id = "vault_s${i}"
}

service_registration "consul" {
  address = "consul-server-0:8500"
}
EOF
done

### Vault Performance Nodes docker-compose up

In [ ]:
# Restart Vault Cluster
docker-compose -f docker-compose-hashi.yml up --force-recreate -d \
  vault_s4

In [ ]:
docker exec -i vault_s4 sh -s <<EOM
export VAULT_ADDR=http://127.0.0.1:8200
vault operator raft join -non-voter http://vault_s1:8200
EOM

In [ ]:
export unseal_key=$(cat /tmp/vault.init | jq -r '.unseal_keys_b64[0]')
printf "${unseal_key}\n"

for i in {4..4}; do
docker exec -i vault_s${i} sh <<EOM
export VAULT_ADDR=http://localhost:8200
vault operator unseal ${unseal_key}
EOM
done

In [ ]:
vault operator raft list-peers

* `vault_s4` is not a voter.

## Vault Replication

optional - Vault Performance Nodes
* main difference here is that it does not auto-join
* joining manually as non-voter from CLI
* https://learn.hashicorp.com/tutorials/vault/performance-standbys?in=vault/enterprise

If you want the secondary Vault Cluster to register with a secondary Consul Cluster, the do this [step](#Consul-Federation-Using-WAN-Gossip) as well.

### Create Vault Configuration - Secondary

Change service registration from `consul-server-0` to `consul-server-4` if desired.

In [ ]:
# Create Vault Server Config
for i in {4..6}; do
cat > vault/config/vault_s${i}/server${i}.hcl <<-EOF
# Note: this file will be re-written by script
api_addr     = "http://10.5.0.10${i}:8200"
cluster_addr = "https://10.5.0.10${i}:8201"
disable_mlock = true

# Base Configuration
listener "tcp" {
  address = "0.0.0.0:8200"
  tls_disable = "true"
}

ui = "true"
log_level="INFO"

# Raft configuration
storage "raft" {
  path    = "/vault/file"
  node_id = "vault_s${i}"
}

service_registration "consul" {
  address = "consul-server-3:8500"
}
EOF
done

### Vault DR Nodes docker-compose up

In [ ]:
# Restart Vault Cluster
docker-compose up --force-recreate -d \
  vault_s4 vault_s5 vault_s6

In [ ]:
# Restart Vault Cluster
docker-compose up --force-recreate -d \
  vault_s5

### Init Vault `init.sh` - PR Secondary

In [ ]:
printf "Init vault_s4 \n"
docker exec \
  -e VAULT_ADDR=http://localhost:8200 \
  vault_s4 vault operator init -format=json -n 1 -t 1 > /tmp/vault_secondary.init

In [ ]:
export root_token2=$(jq -r '.root_token' /tmp/vault_secondary.init)
printf "Root VAULT TOKEN 2 is: $root_token2 \n"

export unseal_key2=$(cat /tmp/vault_secondary.init | jq -r '.unseal_keys_b64[0]')
printf "Unseal Key 2 is:       ${unseal_key2}\n"

### Unseal Vault `unseal.sh` - PR Secondary

In [ ]:
for i in {4..4}; do
docker exec -i vault_s${i} sh <<EOM
export VAULT_ADDR=http://localhost:8200
vault operator unseal ${unseal_key2}
EOM
done

### Verify Vault

In [ ]:
docker exec -i vault_s4 sh <<EOM
export VAULT_TOKEN=$root_token2
vault token lookup
vault status
vault version
vault operator raft list-peers
EOM

### Enable Replication on Primary - PR and DR

In [ ]:
export VAULT_TOKEN=${VAULT_TOKEN_PRIMARY}
printf "#==> Enable Performance Replication as primary\n"
vault write -f sys/replication/performance/primary/enable
printf "#==> Enable Disaster Replication as secondary\n"
vault write -f sys/replication/dr/primary/enable

Revoke **Performance** Secondary Token

In [ ]:
vault write sys/replication/performance/primary/revoke-secondary id=perfsec || true

In [ ]:
printf "#==> Generate a PR secondary token.\n"
vault write -field wrapping_token sys/replication/performance/primary/secondary-token id=perfsec \
  > /tmp/secondaryToken.out && cat /tmp/secondaryToken.out

In [ ]:
secondaryToken=$(cat /tmp/secondaryToken.out)

Revoke **DR** Secondary Token

In [ ]:
vault write sys/replication/dr/primary/revoke-secondary id=drsec || true

Generate **DR** Secondary Token

In [ ]:
printf "#==> Generate a DR secondary token.\n"
vault write -field wrapping_token sys/replication/dr/primary/secondary-token id=drsec \
  > /tmp/drSecondaryToken.out && cat /tmp/drSecondaryToken.out

drSecondaryToken=$(cat /tmp/drSecondaryToken.out)

### Enable Replication on Secondary - PR

In [ ]:
echo $root_token2
echo $secondaryToken

In [ ]:
docker exec -i vault_s4 sh <<EOM
printf "#==> Enable Performance Replication as secondary\n"
export VAULT_TOKEN=$root_token2
vault write sys/replication/performance/secondary/enable token=$secondaryToken
EOM

In [ ]:
printf "\n#==> Status of PR Primary\n"
vault read sys/replication/performance/status || true
printf "\n#==> Status of PR Secondary\n"
docker exec vault_s4 sh -c "vault read sys/replication/performance/status || true"
printf "\n#==> Status of DR Primary\n"
vault read sys/replication/dr/status || true
printf "\n#==> Status of DR Secondary\n"
docker exec vault_s5 sh -c "vault read sys/replication/dr/status || true"

### DR Secondary

#### Init Vault `init.sh` - DR Secondary

In [ ]:
printf "Init vault_s5 \n"
docker exec \
  -e VAULT_ADDR=http://localhost:8200 \
  vault_s5 vault operator init -format=json -n 1 -t 1 > /tmp/vault_drsecondary.init

In [ ]:
export root_token3=$(jq -r '.root_token' /tmp/vault_drsecondary.init)
printf "Root VAULT TOKEN 3 is: $root_token3 \n"

export unseal_key3=$(cat /tmp/vault_drsecondary.init | jq -r '.unseal_keys_b64[0]')
printf "Unseal Key 3 is:       ${unseal_key3}\n"

#### Unseal Vault `unseal.sh` - DR Secondary

In [ ]:
for i in {5..5}; do
docker exec -i vault_s${i} sh <<EOM
export VAULT_ADDR=http://localhost:8200
vault operator unseal ${unseal_key3}
EOM
done

#### Verify Vault - DR Secondary

In [ ]:
docker exec -i vault_s5 sh <<EOM
export VAULT_TOKEN=$root_token3
vault token lookup
vault status
vault version
vault operator raft list-peers
EOM

### Enable Replication on Secondary - DR

In [ ]:
echo $root_token3
echo $drSecondaryToken

In [ ]:
docker exec -i vault_s5 sh <<EOM
printf "#==> Enable DR as secondary\n"
export VAULT_TOKEN=$root_token3
vault write sys/replication/dr/secondary/enable token=$drSecondaryToken
EOM

In [ ]:
printf "\n#==> Status of PR Primary\n"
vault read sys/replication/performance/status || true
printf "\n#==> Status of PR Secondary\n"
docker exec vault_s4 sh -c "vault read sys/replication/performance/status || true"
printf "\n#==> Status of DR Primary\n"
vault read sys/replication/dr/status || true
printf "\n#==> Status of DR Secondary\n"
docker exec vault_s5 sh -c "vault read sys/replication/dr/status || true"

### Promote Secondary - DR

#### Batch Token for Replication Operations

Create a policy named "`dr-secondary-promotion`".

https://learn.hashicorp.com/tutorials/vault/disaster-recovery#dr-operation-token-strategy

In [ ]:
vault policy write dr-secondary-promotion - <<EOF
path "sys/replication/dr/secondary/promote" {
  capabilities = [ "update" ]
}

# To update the primary to connect
path "sys/replication/dr/secondary/update-primary" {
    capabilities = [ "update" ]
}

# Only if using integrated storage (raft) as the storage backend
# To read the current autopilot status
path "sys/storage/raft/autopilot/state" {
    capabilities = [ "update" , "read" ]
}
EOF

Create a token role named "`failover-handler`" with the `dr-secondary-promotion` policy attached and its type should be `batch`.

In [ ]:
vault write auth/token/roles/failover-handler \
    allowed_policies=dr-secondary-promotion \
    orphan=true \
    renewable=false \
    token_type=batch

Create a token for role, "`failover-handler`" with time-to-live (TTL) set to 8 hours.

In [ ]:
vault token create -format=json -role=failover-handler -ttl=8h | tee /tmp/bToken.out | jq .auth
bToken=$(jq -r .auth.client_token /tmp/bToken.out)

Promote the DR secondary (Cluster B) to become the new primary. The request must pass the DR operation token.

In [ ]:
docker exec vault_s5 \
  sh -c "vault write sys/replication/dr/secondary/promote dr_operation_token=$bToken"

Sample Output:
```
WARNING! The following warnings were returned from Vault:

  * This cluster is being promoted to a replication primary. Vault will be
  unavailable for a brief period and will resume service shortly.
```

## haproxy - Load Balancer

In this section, we will set up haproxy to provide performance and high-availability for Vault. Client requests sent to haproxy for Vault will treated the following way:

* `GET` requests will be round-robined to all Vault nodes
* Non-`GET` requests will be sent only to the active Vault node.

### Prerequisites

* [Vault Cluster](#Vault-Setup---Primary)

### Bring up the load balancer

In [ ]:
docker-compose -f docker-compose-hashi.yml \
  -f docker-compose-proxy.yml up --force-recreate -d \
  haproxy

### Validate

In [ ]:
export VAULT_TOKEN=$(cat /tmp/vault.init | jq -r '.root_token')

#### Write test

Send POST (Write) request - Should go to "active" backend

In [ ]:
curl -H "X-Vault-Token: ${VAULT_TOKEN}" \
  -X POST \
  -d '{"data":{"foo":"bar"}}' \
  http://127.0.0.1:18200/v1/kv/data/game/account | jq -c
docker logs haproxy 2>&1 | tail -n 1

This should go to active server from `primary_cluster_active_api` backend.
```
... primary_cluster_api primary_cluster_active_api/vault-active 0/1/252 389 -- 1/1/0/0/0 0/0
```

#### Read test

Send several GET (Read) requests - Should go to different nodes in "read" backend.

In [ ]:
for i in {1..9}; do
curl -s -H "X-Vault-Token: ${VAULT_TOKEN}" \
  -X GET \
  http://127.0.0.1:18200/v1/kv/data/game/account | jq -c .data.data
docker logs haproxy 2>&1 | tail -n 1
done

This should go to any server from `vault_read` backend
```
... primary_cluster_api vault_read/vault_s2 0/0/6 423 -- 1/1/0/0/0 0/0
```

More info:

* https://learn.hashicorp.com/tutorials/consul/load-balancing-haproxy

### Reload haproxy

If you make changes to `haproxy.cfg`, you can reload `haproxy`.

In [ ]:
docker kill -s HUP haproxy

# Clean Up

If you are done with your tests, you might want to shut everything down to reduce your heating bills.

### docker-compose down - everything

In [ ]:
CONSUL_DC=west
CONSUL_DC_2=east
export COMPOSE_PROJECT_NAME=hashi
export COMPOSE_FILE=docker-compose-hashi.yml:docker-compose-proxy.yml:docker-compose-vault.yml:docker-compose.yml

In [182]:
docker-compose down

Stopping consul-server-0 ... 
Stopping consul-server-1 ... 
Stopping consul-server-2 ... 
Stopping grafana         ... 
Stopping cadvisor        ... 
Stopping prometheus      ... 
Stopping node-exporter   ... 
Stopping vault_s3        ... 
Stopping vault_s2        ... 
Stopping vault_s1        ... 
Removing consul-server-3 ... mdone
Removing consul-server-5 ... 
Removing consul-server-4 ... 
Removing consul-server-0 ... 
Removing consul-server-1 ... 
Removing consul-server-2 ... 
Removing consul-agent-1  ... 
Removing grafana         ... 
Removing cadvisor        ... 
Removing prometheus      ... 
Removing node-exporter   ... 
Removing vault_s3        ... 
Removing vault_s2        ... 
Removing vault_s1        ... 
Removing network hashi_defaultdone
Removing network hashi_vpcbr


## DEBUGGING

### Review logs

Review consul logs - for docker

In [ ]:
for i in {0..3}; do
printf "docker logs consul-server-${i}\n"
docker logs consul-server-${i} | { head ; tail -n 3;}
printf "\n"
done

# Appendix

## Addresses


| Name | Address | Description |
| :--- | --- | --- |
| Consul | http://192.168.17.101:8500 | Consul Dashboard
| Nomad | http://192.168.17.101:4646 | Nomad Dashboard
| Vault | http://192.168.17.101:8200 | Vault Dashboard
| haproxy stats | http://192.168.17.101:11936 | haproxy Consul Dashboard
| haproxy - Consul | http://192.168.17.101:18500 | haproxy Consul Dashboard
| haproxy - Nomad | http://192.168.17.101:14646 | haproxy Nomad Dashboard
| haproxy - Vault | http://192.168.17.101:18200 | haproxy Vault Dashboard
| | |
| demo-webapp | http://192.168.17.101:8080 | web - shows db creds
| Consul 2 | http://192.168.17.101:8520 | Consul Dashboard
| Vault PR Secondary | http://192.168.17.101:8210 | Vault Dashboard
| Vault DR Secondary | http://192.168.17.101:8220 | Vault Dashboard
| Prometheus | http://192.168.17.101:9090<br>http://192.168.17.101:9090/targets | Prometheus Dashboard
| Grafana | http://192.168.17.101:3000 | Grafana Dashboard
| cadvisor | http://192.168.17.101:8080<br>http://192.168.17.101:8080/docker/consul-server-0<br>http://192.168.17.101:8080/docker/vault_s1| cAdvisor Dashboard

## Passwords

In [ ]:
printf "\n*** Please Run: export VAULT_TOKEN=${VAULT_TOKEN} \n"

## Directory Structure

In [ ]:
tree

Sample Output for tree
<details><summary></summary>

```
.
├── README.md
├── consul
│   ├── cert
│   │   ├── client
│   │   │   ├── consul-agent-ca.pem
│   │   │   ├── west-client-consul-0-key.pem
│   │   │   └── west-client-consul-0.pem
│   │   └── server
│   │       ├── consul-agent-ca.pem
│   │       ├── consul.hclic
│   │       ├── west-server-consul-0-key.pem
│   │       └── west-server-consul-0.pem
│   ├── config
│   │   ├── acl.hcl
│   │   ├── server.hcl
│   │   ├── server0.hcl
│   │   ├── server1.hcl
│   │   ├── server2.hcl
│   │   ├── server4.hcl
│   │   └── server5.hcl
│   └── policies
├── consul-agent-ca-key.pem
├── consul-agent-ca.pem
├── docker
│   └── haproxy
│       └── haproxy.cfg
├── docker-compose-app.yml
├── docker-compose-consul-app.yml
├── docker-compose-hashi.yml
├── docker-compose-proxy.yml
├── docker-compose-scratch.yml
├── docker-compose.yml
├── grafana
│   ├── dashboards
│   │   ├── alerts.yaml
│   │   ├── consul-server-monitoring_rev3.json
│   │   ├── node-exporter-full_rev22.json
│   │   ├── rules.yaml
│   │   ├── tempo-operational.json
│   │   ├── tempo-reads.json
│   │   ├── tempo-resources.json
│   │   └── tempo-writes.json
│   └── provisioning
│       ├── dashboards
│       │   └── dashboards.yaml
│       └── datasources
│           └── datasource.yml
├── haproxy
│   ├── haproxy.cfg
│   └── haproxy.cfg.txt2
├── hashi_troubleshooting.ipynb
└── vault
    ├── config
    │   ├── vault_s1
    │   │   └── server1.hcl
    │   ├── vault_s2
    │   │   └── server2.hcl
    │   └── vault_s3
    │       └── server3.hcl
    └── logs
        ├── vault_s1
        ├── vault_s2
        ├── vault_s3
        └── vaults_s3
```
</details>

## Resources

* https://learn.hashicorp.com/tutorials/consul/deployment-guide

Vault
* [Vault DR Operation Token Strategy](https://learn.hashicorp.com/tutorials/vault/disaster-recovery#dr-operation-token-strategy)

# Advanced Use Cases

## Consul Auto Upgrade

Modify `docker-compose-hashi.yml`. For consul-server-3, 4, and 5, comment the image parameter for `1.9` and uncomment the one for `latest`.

```yaml
    # image: hashicorp/consul-enterprise:1.9-ent
    image: hashicorp/consul-enterprise:latest
```

Start consul-server-3 consul-server-4 consul-server-5

In [ ]:
docker-compose -f docker-compose-hashi.yml up --force-recreate -d \
  consul-server-3 consul-server-4 consul-server-5

Verify Consul

In [ ]:
printf "#==> List Members\n"
consul members
# curl http://127.0.0.1:8500/v1/agent/members | jq -c .[]
printf "\n#==> List Raft Peers\n"
consul operator raft list-peers

## Consul Redundancy Zones

> NOTE: This is an Enterprise only feature.

You will configure fault resiliency for Consul using redundancy zones.

Redunancy zones is a Consul autopilot feature that makes it possible to run:
* **one voter** and any number of non-voters in each defined zone.

You will set up one voter and one non-voter in three regions.
* If one zone is completely lost, both the the voter and non-voters will be lost.
  * However, the the cluster will remain available.
* If only the voter is lost in a zone, autopilot will promote the non-voter to voter automatically.
  * Puts the hot standby server into service quickly.

You will implement isolated failure domains such as AWS Availability Zones to obtain redundancy within an AZ with less overhead sustained by a larger quorum.

#### Prerequisites

You will need:
* A Consul Enterprise cluster with three servers. See `Consul Setup`.
* Three extre nodes to be used as non-voters.

#### Create Consul config for Redundancy Zone.

This is for the three servers currently running. 

In [152]:
for i in {0..2}; do
docker exec -i consul-server-${i} sh <<EOM
cat > /consul/config/rz.hcl <<EOF
node_meta {
  zone = "zone${i}"
}
EOF
cat /consul/config/rz.hcl
consul reload
EOM
done

node_meta {
  zone = "zone0"
}
Configuration reload triggered
node_meta {
  zone = "zone1"
}
Configuration reload triggered
node_meta {
  zone = "zone2"
}
Configuration reload triggered


* `node_meta` allows us to add a tag `zone` to a server
* `consul reload` triggers a reload of the configuration files. 

Verify the configuration is in place using the `/agent/self` API endpoint.

In [153]:
for i in {0..2}; do
docker exec consul-server-${i} \
curl -s localhost:8500/v1/agent/self | jq ". | .Config, .Meta"
done

{
  "Datacenter": "west",
  "NodeName": "consul-server-0",
  "NodeID": "c7a2981e-f3d4-2958-7fbb-45fa3386c1a2",
  "Revision": "3879c342",
  "Server": true,
  "Version": "1.9.11+ent"
}
{
  "consul-network-segment": "",
  "zone": "zone0"
}
{
  "Datacenter": "west",
  "NodeName": "consul-server-1",
  "NodeID": "50fd4d2c-edc9-ee9b-9384-57a86c4b5ec2",
  "Revision": "3879c342",
  "Server": true,
  "Version": "1.9.11+ent"
}
{
  "consul-network-segment": "",
  "zone": "zone1"
}
{
  "Datacenter": "west",
  "NodeName": "consul-server-2",
  "NodeID": "04b81f7e-c87c-57f8-eb44-9b3939a3e490",
  "Revision": "3879c342",
  "Server": true,
  "Version": "1.9.11+ent"
}
{
  "consul-network-segment": "",
  "zone": "zone2"
}


We check all three servers. We use `docker exec` since only one server container is exposing ports.

Expected Output
```json
...
{
  "consul-network-segment": "",
  "zone": "zone0"
}
...
```

#### Update Consul autopilot configuration

Update Consul autopilot configuration so it knows which `node_meta` tag is used for `-redundancy-zone-tag`.

In [154]:
#// Confirm nothing is currently set.
consul operator autopilot get-config | grep Redundancy

RedundancyZoneTag = ""


```
RedundancyZoneTag = ""
```

In [155]:
consul operator autopilot set-config -redundancy-zone-tag=zone

Configuration updated!


#### Verify autopilot updates

In [156]:
consul operator autopilot get-config

CleanupDeadServers = true
LastContactThreshold = 200ms
MaxTrailingLogs = 250
MinQuorum = 0
ServerStabilizationTime = 10s
RedundancyZoneTag = "zone"
DisableUpgradeMigration = false
UpgradeVersionTag = ""


Sample Output
```shell
CleanupDeadServers = true
LastContactThreshold = 200ms
MaxTrailingLogs = 250
MinQuorum = 0
ServerStabilizationTime = 10s
RedundancyZoneTag = "zone"   <==---
DisableUpgradeMigration = false
UpgradeVersionTag = ""
```

#### Create Consul config - Redundancy Zone for new nodes

In [157]:
for i in {0..2}; do
cat > consul/config/rz-${i}.hcl <<-EOF
node_meta {
  zone = "zone${i}"
}
autopilot {
  redundancy_zone_tag = "zone"
}
EOF
done

Server 3, 4, and 5 are used for various scenarios. In this scenario, we can reuse the configs from the first cluster.

In [158]:
cp consul/config/server.hcl consul/config/server_dc2.hcl

Click here if you want to view the config files.
* [server.hcl](./consul/config/server.hcl)
* [server_dc2.hcl](./consul/config/server_dc2.hcl)

Modify [docker-compose-hashi.yml](docker-compose-hashi.yml). For `consul-server-3`, uncomment the image parameter for `1.9` and comment the one for `latest`. Servers 4 and 5 will inherit the settings.

```yaml
    image: hashicorp/consul-enterprise:1.9-ent
    # image: hashicorp/consul-enterprise:latest
```

#### Bring up new Consul nodes

Start consul-server-3 consul-server-4 consul-server-5

In [181]:
CONSUL_DC=west CONSUL_DC_2=west
docker-compose up --force-recreate -d \
  consul-server-3 consul-server-4 consul-server-5

Recreating consul-server-3 ... 
Recreating consul-server-4 ... 
Recreating consul-server-5 ... 
eating consul-server-3 ... done

Verify Consul with `operator` subcommand.

In [162]:
printf "#==> List Members\n"
consul members
# curl http://127.0.0.1:8500/v1/agent/members | jq -c .[]
printf "\n#==> List Raft Peers\n"
consul operator raft list-peers

#==> List Members
Node             Address         Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301   alive   server  1.9.11+ent  2         west  <all>
App1             10.5.0.12:8301  alive   client  1.9.11+ent  2         west  <default>

#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  c7a2981e-f3d4-2958-7fbb-45fa3386c1a2  10.5.0.2:8300  follower  true   3
consul-server-2  04b81f7e-c87c-57f8-eb44-9b3939a3e490  10.5.0.4:8300  leader    true   3
consul-server-1  50fd4d2c-edc9-ee9b-9384-57a86c4b5ec2  10.5.0.3:8300  follower  true   3


Sample Output
```
#==> List Members
Node             Address         Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-3  10.5.0.7:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-4  10.5.0.6:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-5  10.5.0.5:8301   alive   server  1.9.11+ent  2         west  <all>
App1             10.5.0.12:8301  alive   client  1.9.11+ent  2         west  <default>

#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  follower  true   3
consul-server-1  d7e82aa0-2fa0-9308-5970-44e839786d2b  10.5.0.3:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-5  b9c2c042-b161-3085-f417-d5739e6cbb50  10.5.0.5:8300  follower  false  3
consul-server-4  21c6c18d-b89d-edca-4f8e-011cbb244036  10.5.0.6:8300  follower  false  3
consul-server-3  524afd65-feea-79f2-1431-09b6dbbb8c05  10.5.0.7:8300  follower  false  3
```

* **NOTE:** All the new servers, once started, are added to the datacenter as non-voters (`Voter` = `false`). You can reference the Voter column in the output to verify it.

#### Test fault tolerance

Stop one of the voters. We use `consul-server-1` from `zone1`.

In [ ]:
docker stop consul-server-1

Verify that the correspondent non-voter in its redundancy zone gets promoted as a voter as soon as the server gets declared unhealthy.

In [ ]:
consul operator raft list-peers

```shell
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-5  b9c2c042-b161-3085-f417-d5739e6cbb50  10.5.0.5:8300  follower  false  3
consul-server-4  21c6c18d-b89d-edca-4f8e-011cbb244036  10.5.0.6:8300  follower  true   3   <==---
consul-server-3  524afd65-feea-79f2-1431-09b6dbbb8c05  10.5.0.7:8300  follower  false  3
```

* `consul-server-4` from `zone1` is now a voter

Once `server-server-4` gets promoted as a voter you can start Consul on `consul-server-1` again and verify the one voter per redundancy zone rule is still respected.

In [ ]:
docker start consul-server-1

In [ ]:
consul operator raft list-peers

```shell
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-5  b9c2c042-b161-3085-f417-d5739e6cbb50  10.5.0.5:8300  follower  false  3
consul-server-4  21c6c18d-b89d-edca-4f8e-011cbb244036  10.5.0.6:8300  follower  true   3
consul-server-3  524afd65-feea-79f2-1431-09b6dbbb8c05  10.5.0.7:8300  follower  false  3
consul-server-1  d7e82aa0-2fa0-9308-5970-44e839786d2b  10.5.0.3:8300  follower  false  3   <==---
```

**NOTE:** `consul-server` is up as a `follower`, but is no longer a `voter`.

If you no longer need these nodes you can stop them.

Stop consul-server-3 consul-server-4 consul-server-5

In [ ]:
for i in {3..5}; do
echo "#==> Stopping consul-server-${i}"
docker-compose stop consul-server-${i}
consul operator raft list-peers
sleep 2
done

In [ ]:
consul operator raft list-peers

#### Troubleshooting

```
Failed to join 10.5.0.2: No installed keys could decrypt the message
```

* https://learn.hashicorp.com/tutorials/consul/gossip-encryption-rotate

Create new keys

In [164]:
export CONSUL_HTTP_ADDR="http://localhost:8500"
export NEW_KEY=`consul keygen`
echo $NEW_KEY

for i in {0..5}; do
docker exec -i consul-server-${i} sh <<EOM
# Install the key
consul keyring -install ${NEW_KEY}

# Set as primary
consul keyring -use ${NEW_KEY}
EOM
done

Ayv87NXKJMGNSGzpmnRMsWoZH97AwiCsNkCMyB0q90E=
==> Installing new gossip encryption key...
==> Changing primary gossip encryption key...
==> Installing new gossip encryption key...
==> Changing primary gossip encryption key...
==> Installing new gossip encryption key...
==> Changing primary gossip encryption key...
==> Installing new gossip encryption key...
==> Changing primary gossip encryption key...
==> Installing new gossip encryption key...
==> Changing primary gossip encryption key...
==> Installing new gossip encryption key...
==> Changing primary gossip encryption key...


Delete old keys

In [ ]:
for i in {0..5}; do
docker exec -i \
  -e CONSUL_HTTP_ADDR=$CONSUL_HTTP_ADDR \
  -e NEW_KEY=$NEW_KEY \
  consul-server-${i} \
  sh -s <<"EOM"
echo "#==> Retrieve all keys used by Consul"
echo "Host: $(hostname)"
KEYS=$(curl -s http://localhost:8500/v1/operator/keyring)
#echo Keys: $KEYS #DEBUGGING 
ALL_KEYS=$(echo ${KEYS} | jq -r '.[].Keys| to_entries[].key' | sort | uniq)

echo "#==> Delete all older keys used by Consul"
for i in `echo ${ALL_KEYS}`; do
  # echo $i #DEBUGGING
  if [ $i != ${NEW_KEY} ] ; then
    echo consul keyring -remove $i
    consul keyring -remove $i
  fi
done
EOM
done

`last_log_index` and `commit_index`


## Consul Federation Using WAN Gossip

### Create Server Configuration - DC2

Create Core Consul config - Server

In [118]:
#debugging - in case variable from earlier was lost
export CONSUL_KEY="Y+rrmAn0c9R7MLIf/eRNOjvglJA+z9dY/uqyqitaB0E="

In [119]:
# for i in {3..5}; do
tee consul/config/server_dc2.hcl <<-EOF
# datacenter  = "${CONSUL_DC_2}" # in CLI
# node_name   = "ConsulServer${i}" # in CLI
bind_addr   = "0.0.0.0" #default
client_addr = "0.0.0.0" #default 127.0.0.1
data_dir    = "/consul/data"
log_level   = "DEBUG"

encrypt     = "${CONSUL_KEY}"
ca_file     = "/consul/cert/consul-agent-ca.pem"
cert_file   = "/consul/cert/${CONSUL_DC_2}-server-consul-0.pem"
key_file    = "/consul/cert/${CONSUL_DC_2}-server-consul-0-key.pem"
verify_incoming = true
verify_outgoing = true
verify_server_hostname = true

# server           =  true # in CLI
bootstrap_expect = 1
retry_join  = [ "consul-server-3", "consul-server-4", "consul-server-5" ]
ui_config { enabled = true } 

#// 5 is default multiplier
performance {
  raft_multiplier = 2 #// fast but not too fast
}

discovery_max_stale = "5s"

telemetry {
    prometheus_retention_time = "8h",
    disable_hostname = true
}

connect {
    enabled = true
}

enable_local_script_checks = true
EOF
# done

# datacenter  = "east" # in CLI
# node_name   = "ConsulServer5" # in CLI
bind_addr   = "0.0.0.0" #default
client_addr = "0.0.0.0" #default 127.0.0.1
data_dir    = "/consul/data"
log_level   = "DEBUG"

encrypt     = "Y+rrmAn0c9R7MLIf/eRNOjvglJA+z9dY/uqyqitaB0E="
ca_file     = "/consul/cert/consul-agent-ca.pem"
cert_file   = "/consul/cert/east-server-consul-0.pem"
key_file    = "/consul/cert/east-server-consul-0-key.pem"
verify_incoming = true
verify_outgoing = true
verify_server_hostname = true

# server           =  true # in CLI
bootstrap_expect = 1
retry_join  = [ "consul-server-3", "consul-server-4", "consul-server-5" ]
ui_config { enabled = true } 

#// 5 is default multiplier
performance {
  raft_multiplier = 2 #// fast but not too fast
}

discovery_max_stale = "5s"

telemetry {
    prometheus_retention_time = "8h",
    disable_hostname = true
}

connect {
    enabled = true
}

enable_local_script_checks = true


### Consul docker-compose up

We will now bring up the three Consul servers in a second Consul Cluster. You can use `--force-recreate` to have Docker recreate the containers.

In [120]:
export CONSUL_DC=west CONSUL_DC_2=east
docker-compose \
  up --force-recreate -d \
  consul-server-3 consul-server-4 consul-server-5

Recreating consul-server-5 ... 
Recreating consul-server-4 ... 
Recreating consul-server-3 ... 
eating consul-server-5 ... done

### Verify Consul

Quick check to make sure your Consul environment is running correctly.

In [121]:
docker exec -i consul-server-3 sh <<EOM
echo export CONSUL_HTTP_TOKEN=$CONSUL_HTTP_TOKEN
printf "#==> List Members\n"
consul members
# curl http://127.0.0.1:8500/v1/agent/members | jq -c .[]
printf "\n#==> List Raft Peers\n"
consul operator raft list-peers
printf "\n#==> List services from Consul catalog\n"
consul catalog services
EOM

export CONSUL_HTTP_TOKEN=1eb8a07d-3b5a-9069-2673-e031a48851a7
#==> List Members
Node             Address        Status  Type    Build       Protocol  DC    Segment
consul-server-3  10.5.1.2:8301  alive   server  1.9.11+ent  2         east  <all>
consul-server-4  10.5.1.3:8301  alive   server  1.9.11+ent  2         east  <all>
consul-server-5  10.5.1.4:8301  alive   server  1.9.11+ent  2         east  <all>

#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-4  7cc77c31-896f-3a61-acb7-c6fb79444d93  10.5.1.3:8300  leader    true   3
consul-server-3  c8de739c-98f9-aa81-a33e-e6eade88a80b  10.5.1.2:8300  follower  true   3
consul-server-5  f4cb804b-619a-60e3-4245-bfe4ee0dc998  10.5.1.4:8300  follower  true   3

#==> List services from Consul catalog
consul


You should see something like the following.
* There should be three servers. `DC` should match
```
#==> List Members
Node             Address        Status  Type    Build       Protocol  DC    Segment
consul-server-3  10.5.1.2:8301  alive   server  1.10.4+ent  2         east  <all>
consul-server-4  10.5.1.3:8301  alive   server  1.10.4+ent  2         east  <all>
consul-server-5  10.5.1.4:8301  alive   server  1.10.4+ent  2         east  <all>
```

* There should be a leader and two followers.

```
#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-3  cff9fe90-1bfe-84a9-72cb-84f132297c32  10.5.1.2:8300  leader    true   3
consul-server-4  7b7d6e78-fc82-9ea7-4c99-e17d2be86439  10.5.1.3:8300  follower  true   3
consul-server-5  dd9fc878-7cd5-3b86-a122-bb2000c446f2  10.5.1.4:8300  follower  true   3
```


```
#==> List services from Consul catalog
consul
```

In [112]:
for i in {3..5}; do
docker logs consul-server-${i} | (head; tail -n 5)
done

==> Starting Consul agent...
           Version: '1.9.11+ent'
           Node ID: 'c8de739c-98f9-aa81-a33e-e6eade88a80b'
         Node name: 'consul-server-3'
        Datacenter: 'east' (Segment: '<all>')
            Server: true (Bootstrap: true)
       Client Addr: [0.0.0.0] (HTTP: 8500, HTTPS: -1, gRPC: -1, DNS: 8600)
      Cluster Addr: 10.5.1.2 (LAN: 8301, WAN: 8302)
           Encrypt: Gossip: false, TLS-Outgoing: true, TLS-Incoming: true, Auto-Encrypt-TLS: false

2021-12-08T00:47:27.208Z [DEBUG] agent.server.serf.lan: serf: messageUserEventType: consul:new-leader
2021-12-08T00:47:27.318Z [DEBUG] agent.server.raft: accepted connection: local-address=10.5.1.2:8300 remote-address=10.5.1.3:49794
2021-12-08T00:47:27.685Z [DEBUG] agent: Skipping remote check since it is managed automatically: check=serfHealth
2021-12-08T00:47:27.685Z [DEBUG] agent: Node info in sync
2021-12-08T00:47:27.685Z [DEBUG] agent: Node info in sync
==> Starting Consul agent...
           Version: '1.9.11+ent'


### Confirm the Datacenters are Joined and Replicating Tokens

In [122]:
consul members -wan

Node                  Address        Status  Type    Build       Protocol  DC    Segment
consul-server-0.west  10.5.0.2:8302  alive   server  1.9.11+ent  2         west  <all>
consul-server-1.west  10.5.0.3:8302  alive   server  1.9.11+ent  2         west  <all>
consul-server-2.west  10.5.0.4:8302  alive   server  1.9.11+ent  2         west  <all>


Join the Datacenters

In [123]:
consul join -wan consul-server-3

Successfully joined cluster by contacting 1 nodes.


#### Verify Consul Federation

Once the join is complete, the members command can be used to verify that all server nodes gossiping over WAN.

In [124]:
consul members -wan

Node                  Address        Status  Type    Build       Protocol  DC    Segment
consul-server-0.west  10.5.0.2:8302  alive   server  1.9.11+ent  2         west  <all>
consul-server-1.west  10.5.0.3:8302  alive   server  1.9.11+ent  2         west  <all>
consul-server-2.west  10.5.0.4:8302  alive   server  1.9.11+ent  2         west  <all>
consul-server-3.east  10.5.1.2:8302  alive   server  1.9.11+ent  2         east  <all>
consul-server-4.east  10.5.1.3:8302  alive   server  1.9.11+ent  2         east  <all>
consul-server-5.east  10.5.1.4:8302  alive   server  1.9.11+ent  2         east  <all>


In [125]:
curl http://localhost:8500/v1/catalog/datacenters

["west","east"]

Check that ACL replication is configured properly in the secondary datacenter.

In [133]:
docker exec -i consul-server-0 sh <<EOF
echo "#==> From dc1"
apk add curl
curl http://localhost:8500/v1/acl/replication?pretty
printf "\n\n"; echo "#==> From dc2"
curl http://consul-server-3:8500/v1/acl/replication?pretty
EOF

#==> From dc1
OK: 14 MiB in 30 packages
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    20  100    20    0     0   3894      0 --:--:-- --:--:-- --:--:--  5000
ACL support disabled

#==> From dc2
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    20  100    20    0     0   4758      0 --:--:-- --:--:-- --:--:--  6666
ACL support disabled

Query the nodes in each datacenter

In [ ]:
docker exec consul-server-0 \
  curl -s http://localhost:8500/v1/catalog/nodes?dc=${CONSUL_DC} | jq -c .[]

In [ ]:
docker exec consul-server-0 \
  curl -s http://localhost:8500/v1/catalog/nodes?dc=${CONSUL_DC_2} | jq -c .[]

Query for service in both datacenters - `consul.service.west.consul` and `consul.service.east.consul` 

In [ ]:
dig @127.0.0.1 -p 8600 consul.service.west.consul | grep -A3 "ANSWER SECTION"

In [ ]:
dig @127.0.0.1 -p 8600 consul.service.east.consul | grep -A3 "ANSWER SECTION"

NOTES:

* All server nodes must be able to talk to each other; Else gossip and RPC forwarding will not work
* Data is not replicated between Consul Clusters
  * Request made for resource in another datacenter is forwarded to remote Consul Servers
  

### Additional Consul Steps

Setup Consul environment variables - Notice that since TLS encryption is enabled, you will now need to use the server certificates to complete all other tasks.

In [ ]:
export CONSUL_CACERT=/etc/consul.d/consul-agent-ca.pem
export CONSUL_CLIENT_CERT=/etc/consul.d/<dc-name>-<server/ client>-consul-<cert-number>.pem
export CONSUL_CLIENT_KEY=/etc/consul.d/<dc-name>-<server/   client>-consul-<cert-number>-key.pem

## Vault DR and PR

## Debug - Network

In [14]:
docker exec -i consul-server-0 sh <<"EOM"
hostname
for i in consul-server-1 consul-server-2 consul-agent-1; do
ping -qc 1 ${i}
done
EOM

consul-server-0
PING consul-server-1 (10.5.0.3) 56(84) bytes of data.

--- consul-server-1 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.214/0.214/0.214/0.000 ms
PING consul-server-2 (10.5.0.4) 56(84) bytes of data.

--- consul-server-2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.286/0.286/0.286/0.000 ms
PING consul-agent-1 (10.5.0.12) 56(84) bytes of data.

--- consul-agent-1 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.199/0.199/0.199/0.000 ms


### docker-compose restart

In [ ]:
docker-compose -f docker-compose-hashi.yml restart

### docker-compose down

In [ ]:
docker-compose -f docker-compose-hashi.yml down

In [ ]:
docker-compose stop grafana prometheus

### Restart Vault Cluster

In [ ]:
docker-compose -f docker-compose-hashi.yml restart vault_s1 vault_s2 vault_s3

## Vault DB

In [ ]:
# This script configures a Postgres Dynamic Database credential database for benchmarking
vault secrets enable database

vault write database/config/postgres \
  plugin_name=postgresql-database-plugin \
  allowed_roles="*" \
  connection_url="postgresql://{{username}}:{{password}}@db:5432/products?sslmode=disable" \
  username="postgres" \
  password="password"

vault write database/roles/benchmarking \
    db_name=postgres \
    creation_statements="CREATE ROLE \"{{name}}\" WITH LOGIN PASSWORD '{{password}}' VALID UNTIL '{{expiration}}'; \
        GRANT SELECT ON ALL TABLES IN SCHEMA public TO \"{{name}}\";" \
    default_ttl="24h" \
    max_ttl="48h"

vault read database/creds/benchmarking

Admin token (optional): You may prefer using an admin token instead of root (for example if you’re using an existing cluster). If so, create an admin token using the vault-admin.hcl policy file shown below. This admin policy is authored based on the Vault Policies guide.

In [ ]:
# Assuming that VAULT_TOKEN is set with root or higher Admin token
vault policy write learn-admin admin-policy.hcl
vault token create -policy=learn-adminexport
VAULT_TOKEN=<token-from-above command>
vault token lookup

In [ ]:
consul members
consul operator raft list-peers
consul operator autopilot get-config
vault operator raft list-peers

## Onboarding App

In [ ]:
vault secrets list
vault read database/config/postgres

### Vault Onboarding docker-compose up

We will now bring up the three Consul servers and one client. You can use `--force-recreate` to have Docker recreate the containers.

In [ ]:
export CONSUL_DC=west CONSUL_DC_2=east
docker-compose \
  up --force-recreate -d \
  db web vault-agent haproxy

In [ ]:
git clone https://github.com/hashicorp/vault-guides.git tmp/vault-guides

In [ ]:
cp tmp/vault-guides/operations/onboarding/terraform/*.* terraform/

In [ ]:
cp -r tmp/vault-guides/operations/onboarding/docker-compose/vault-agent .

### Vault administration with Terraform

Modifications: 

* I modified the `auth.tf`. Changed the local file destination since `vault-agent` folder is not under `docker-compose` folder.

In [ ]:
TF_CLI_ARGS="-input=false"

In [ ]:
terraform -chdir=terraform init

In [ ]:
terraform -chdir=terraform plan

In [ ]:
terraform -chdir=terraform apply -auto-approve

In [ ]:
docker restart vault-agent

Access http://localhost:8080 on your browser, and you should be able to see the nginx application display a dynamic PostgreSQL database credential provided by Vault as shown below. Also try accessing http://localhost:8080/kv.html to see example static secret values.



<img src="https://www.datocms-assets.com/2885/1624893789-vtf-onboarding-2.png?fit=max&fm=webp&q=80&w=2500" width=640 />

The Terraform configurations for this demo are described in more detail below along with the corresponding source file names:

* "**Application entity**" — `entity.tf`:
  * Pre-creating the application entity is optional but encouraged.
  * It allows easier auditing and more flexibility in attaching ACL policies.
  1. Please log in to the Vault UI on http://localhost:8200 with the root token
  1. Then click `Access` > `Entities`. You should see two created entities: `nginx` and `app100`.
  1. Clicking into these entities will display
      * an alias for the AppRole authentication method 
      * and the mapped entity ACL policies.
* "Authentication method" — `auth.tf`:
  * This demo uses the AppRole auth method, which is a type of “trusted orchestrator” secure introduction pattern.
  * An authentication method alias links the entity to the AppRole role.
* "ACL policy" — `entity.tf`: We recommend using templated policies to reduce the overhead of policy management.
  * This demo uses two templated policies:
    * `kv_rw_policy` for accessing key-value secrets
    * `postgres_creds_policy` for accessing dynamic Postgres credentials.

These elements are represented as a Terraform graph diagram snippet, shown below:

Terraform graph snippet for authentication, entity, and ACL policy.

<img src="https://www.datocms-assets.com/2885/1624893793-vtf-onboarding-3.png?fit=max&fm=webp&q=80&w=2500" width=640 />

### Application Integration with Vault

Now that the Vault configurations are built, we need the application to log in to Vault using AppRole credentials and fetch a secret. The demo uses Vault Agent to achieve this (see App Integration for more patterns).
Vault Agent workflow.

Vault Agent workflow.

<img src="https://www.datocms-assets.com/2885/1624893810-vtf-onboarding-5.png?fit=max&fm=webp&q=80&w=2500" width=640 />

The file `nginx-vault-agent.hcl` specifies how to authenticate the `nginx` container using AppRole. It also links two template files, `kv.tpl` and `postgres.tpl`, that tell Vault Agent how to render secrets from a KV and Database Secrets Engine respectively.

### Register a service

In [ ]:
docker exec -i consul-server-1 sh <<EOM
cat > /consul/config/webapp.hcl <<EOF
service {
  name = "webapp",
  port = 80,
  check {
    http = "http://demo-webapp",
    interval = "5s"
  }
}
EOM

In [ ]:
docker exec consul-server-1 consul reload

### Onboarding the Next Application

To onboard another application, simply add its name to the default value of the entities variable in `variables.tf` as shown below for `app200`.

# Snippet from variables.tf after adding app200

In [ ]:
cat > terraform/terraform.tfvars <<EOF
entities = [
    "nginx",
    "app100",
    "app200"
]
EOF

Then run `terraform apply` to create the additional Vault configurations for this application:

NOTE: Ensure that `VAULT_TOKEN` was set from before

In [ ]:
terraform -chdir=terraform validate && \
terraform -chdir=terraform apply -auto-approve

Verify from the Vault UI that there is a new entity called `app200` with an alias to the AppRole auth method:

Vault screenshot showing a new app200 entity being added.

Vault screenshot showing a new app200 entity being added.

A new Role ID and Secret ID have also been created, which you can find by running the terraform output command. We can use this to test authentication and secret access as shown below. Note that the Role ID, Secret ID, and Vault token will be unique in your case.

In [ ]:
terraform -chdir=terraform output -json > /tmp/approle_200.txt

In [ ]:
ROLE_ID=$(jq -r .role_ids.value.app200 /tmp/approle_200.txt)
SECRET_ID=$(jq -r .secret_ids.value.app200 /tmp/approle_200.txt)

In [ ]:
Login using AppRole

In [ ]:
vault write -format=json auth/approle/login \
  role_id=${ROLE_ID} \
  secret_id=${SECRET_ID} | tee /tmp/approle_200.token

In [ ]:
VAULT_TOKEN_APP200=$(jq -r .auth.client_token /tmp/approle_200.token) && echo $VAULT_TOKEN_APP200

Read KV secret

In [ ]:
VAULT_TOKEN=${VAULT_TOKEN_APP200} vault kv get kv/app200/static

Sample Output
```
====== Metadata ======
Key              Value
---              -----
created_time     2021-11-16T20:49:58.138197524Z
deletion_time    n/a
destroyed        false
version          1

====== Data ======
Key         Value
---         -----
app         app200
password    cheese
username    app200
```

To de-board an application, simply remove the entity from the same variable and re-rerun `terraform apply`.

Gabe's Envoy Notes

```
sudo apt install -y apt-transport-https gnupg2 curl lsb-release
curl -sL 'https://deb.dl.getenvoy.io/public/gpg.8115BA8E629CC074.key' | sudo gpg --dearmor -o /usr/share/keyrings/getenvoy-keyring.gpg
echo "deb [arch=amd64 signed-by=/usr/share/keyrings/getenvoy-keyring.gpg] https://deb.dl.getenvoy.io/public/deb/ubuntu $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/getenvoy.list
sudo apt update
sudo apt install -y getenvoy-envoy

envoy --version
```

# Grafana Dashboards Vault

https://docs.datadoghq.com/integrations/vault/?tab=host


GC pause time (ns)
{
"viz": "timeseries",
"query": "avg:service.vault_runtime_gc_pause_ns.quantile{$vault_cluster,quantile:0.9} by {host}",
"data_source": "metrics",
"type": "line",
            "style": {
                "palette": "dog_classic",
                "type": "solid",
                "width": "normal"
            }
        }
    ],
    "yaxis": {
        "include_zero": true,
        "max": "auto",
        "scale": "linear",
        "min": "auto",
        "label": ""
    },
    "markers": []
}

In [191]:
cat > /tmp/generate_traffic_vault.sh <<"EOF"
echo $VAULT_TOKEN
vault secrets list
vault secrets enable kv
vault secrets enable -path=kv-peter kv
for i in {1..1}; do
vault secrets enable -path=kv-app-${i} kv > /dev/null || true
done

printf "\n#==> write and read secrets\n"
for i in {1..10}; do
vault kv put kv/game/account-${i} username=foo-${i} password=bar
vault kv get kv/game/account-${i} > /dev/null
vault kv put kv-peter/data/game/account-${i} username=foo-${i} password=bar
vault kv get kv-peter/data/game/account-${i} > /dev/null
done

printf "\n#==> write and read secrets\n"
for i in {1..10}; do
vault kv put kv-app-${i}/game/account-${i} username=foo-${i} password=bar
vault kv get kv-app-${i}/game/account-${i} > /dev/null
done

printf "\n#==> create tokens\n"
for i in {1..10}; do
vault token create \
  -field=token \
  -policy prometheus-metrics
done

vault auth enable userpass || true
for i in {1..10}; do
vault write auth/userpass/users/mitchellh password=foo policies=admins
vault login -method=userpass username=user-${i} password=foo
done
EOF

chmod +x /tmp/generate_traffic_vault.sh

In [205]:
# for i in {1..10}; do
# vault write auth/userpass/users/user-${i} password=foo policies=admins
# done

for i in {1..10}; do
# vault kv list kv-app-${i}/game > /dev/null
vault login -method=userpass username=user-${i} password=foo 2>&1 > /dev/null
done

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

WARNING! The VAULT_TOKEN environment variable is set! This takes

In [192]:
/tmp/generate_traffic_vault.sh

s.6NzYUa3teopZQOn8IEwcGJrf
Path          Type         Accessor              Description
----          ----         --------              -----------
cubbyhole/    cubbyhole    cubbyhole_ba694efd    per-token private secret storage
identity/     identity     identity_77262344     identity store
kv-app-1/     kv           kv_b46c078e           n/a
kv-app-10/    kv           kv_ed0f646e           n/a
kv-app-2/     kv           kv_3fe7a595           n/a
kv-app-3/     kv           kv_f4134347           n/a
kv-app-4/     kv           kv_aff795ab           n/a
kv-app-5/     kv           kv_69355074           n/a
kv-app-6/     kv           kv_076b22ea           n/a
kv-app-7/     kv           kv_93d5792f           n/a
kv-app-8/     kv           kv_a86a30c4           n/a
kv-app-9/     kv           kv_639869e8           n/a
kv-peter/     kv           kv_def6bdec           n/a
kv/           kv           kv_6a64e477           n/a
sys/          system       system_f11e4d28       system endpoints use

: 2